In [2]:
%matplotlib widget

In [3]:
import pandas as pd
import numpy as np
from glob import glob
import os
import datetime
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Input, Concatenate, Dot, Add, ReLU, Activation
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras

from tqdm import tqdm

In [4]:
import matplotlib
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
font_location = '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf'
#font_location = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
# font_location = 'C:/Windows/Fonts/NanumGothic.ttf' # For Windows
fprop = fm.FontProperties(fname=font_location)

**AWS null 처리**

In [48]:
folder = 'data'
file_name = 'AWS_옥산_2016.xlsx'
path = os.path.join(folder, file_name)
# not needed anymore
if False:
    df = pd.read_excel(path)
    df_new = df[df['stn_name'].notna()]
    df_new.to_excel(path, index=False)

**구미보 interpollation**

In [154]:
# not needed anymore
if False:
    file_names = ['보_구미보_2016.xlsx', '보_구미보_2017.xlsx', '보_구미보_2018.xlsx', '보_구미보_2019.xlsx']
    folder = 'data'
    for file_name in file_names:
        path = os.path.join(folder, file_name)
        df = pd.read_excel(path)
        df.loc[df['upstrim_wlv']==0, 'upstrim_wlv'] = np.nan
        df.loc[df['dwstrm_wlv']==0, 'dwstrm_wlv'] = np.nan
        df['upstrim_wlv'].interpolate(limit_direction='both', inplace=True)
        df['dwstrm_wlv'].interpolate(limit_direction='both', inplace=True)
        stn_name = df.loc[df['stn_name'].notna(), 'stn_name'].iloc[0]
        stn_name
        df.loc[df['stn_name'].isna(), 'stn_name'] = stn_name
        df.to_excel(path, index=False)

**GAIN**

**한강**

In [205]:
file_names = [
              ['가평_2013.xlsx',
               '가평_2014.xlsx',
               '가평_2015.xlsx',
               '가평_2016.xlsx',
               '가평_2017.xlsx',
               '가평_2018.xlsx',
               '가평_2019.xlsx',
               '가평_2020.xlsx'],
              ['의암호_2013.xlsx',
               '의암호_2014.xlsx',
               '의암호_2015.xlsx',
               '의암호_2016.xlsx',
               '의암호_2017.xlsx',
               '의암호_2018.xlsx',
               '의암호_2019.xlsx',
               '의암호_2020.xlsx'],
              ['서상_2013.xlsx',
               '서상_2014.xlsx',
               '서상_2015.xlsx',
               '서상_2016.xlsx',
               '서상_2017.xlsx',
               '서상_2018.xlsx',
               '서상_2019.xlsx',
               '서상_2020.xlsx'],
             ]
gan_files = [ '가평2_gan.xlsx', '의암호2_gan.xlsx', '서상2_gan.xlsx']
ori_files = [ '가평2_org.xlsx', '의암호2_org.xlsx', '서상2_org.xlsx']
folder = 'water_data/han/자동'

day = 24*60*60
year = (365.2425)*day

df_full = []
df = []

auto_cols = ['tmpr_value', 'ph_value', 'do_value', 'ec_value', 'toc_value', '총질소_값', '총인_값', '클로로필-a_값']

for loc in range(len(file_names)):
    
    df_loc = []
    for y in range(len(file_names[loc])):
        path = os.path.join(folder, file_names[loc][y])
        print(file_names[loc][y])
        df_loc.append(pd.read_excel(path))
    df_full.append(pd.concat(df_loc))
    df.append(df_full[loc].loc[:, auto_cols].copy())
    date_time = pd.to_datetime(df_full[loc].loc[:, 'ymdh'], format='%Y.%m.%d %H:%M', utc=True)
    timestamp_s = date_time.map(datetime.datetime.timestamp)
    df[loc]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
    df[loc]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
    df[loc]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
    df[loc]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
    df[loc] = df[loc].reset_index(drop=True)

가평_2013.xlsx
가평_2014.xlsx
가평_2015.xlsx
가평_2016.xlsx
가평_2017.xlsx
가평_2018.xlsx
가평_2019.xlsx
가평_2020.xlsx
의암호_2013.xlsx
의암호_2014.xlsx
의암호_2015.xlsx
의암호_2016.xlsx
의암호_2017.xlsx
의암호_2018.xlsx
의암호_2019.xlsx
의암호_2020.xlsx
서상_2013.xlsx
서상_2014.xlsx
서상_2015.xlsx
서상_2016.xlsx
서상_2017.xlsx
서상_2018.xlsx
서상_2019.xlsx
서상_2020.xlsx


In [17]:
file_names = [
              ['가평_2015.xlsx',
               '가평_2016.xlsx',
               '가평_2017.xlsx',
               '가평_2018.xlsx',
               '가평_2019.xlsx',
               '가평_2020.xlsx'],
              ['의암호_2015.xlsx',
               '의암호_2016.xlsx',
               '의암호_2017.xlsx',
               '의암호_2018.xlsx',
               '의암호_2019.xlsx',
               '의암호_2020.xlsx'],
              ['서상_2015.xlsx',
               '서상_2016.xlsx',
               '서상_2017.xlsx',
               '서상_2018.xlsx',
               '서상_2019.xlsx',
               '서상_2020.xlsx'],
             ]
gan_files = [ '가평_gan.xlsx', '의암호_gan.xlsx', '서상_gan.xlsx']
ori_files = [ '가평_org.xlsx', '의암호_org.xlsx', '서상_org.xlsx']
folder = 'water_data/han/자동'

day = 24*60*60
year = (365.2425)*day

df_full = []
df = []

auto_cols = ['tmpr_value', 'ph_value', 'do_value', 'ec_value', 'toc_value', '총질소_값', '총인_값', '클로로필-a_값']

for loc in range(len(file_names)):
    
    df_loc = []
    for y in range(len(file_names[loc])):
        path = os.path.join(folder, file_names[loc][y])
        print(file_names[loc][y])
        df_loc.append(pd.read_excel(path))
    df_full.append(pd.concat(df_loc))
    df.append(df_full[loc].loc[:, auto_cols].copy())
    date_time = pd.to_datetime(df_full[loc].loc[:, 'ymdh'], format='%Y.%m.%d %H:%M', utc=True)
    timestamp_s = date_time.map(datetime.datetime.timestamp)
    df[loc]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
    df[loc]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
    df[loc]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
    df[loc]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
    df[loc] = df[loc].reset_index(drop=True)

가평_2015.xlsx
가평_2016.xlsx
가평_2017.xlsx
가평_2018.xlsx
가평_2019.xlsx
가평_2020.xlsx
의암호_2015.xlsx
의암호_2016.xlsx
의암호_2017.xlsx
의암호_2018.xlsx
의암호_2019.xlsx
의암호_2020.xlsx
서상_2015.xlsx
서상_2016.xlsx
서상_2017.xlsx
서상_2018.xlsx
서상_2019.xlsx
서상_2020.xlsx


In [15]:
(365*6+2)*24

52608

In [206]:
df[0]

,tmpr_value,ph_value,do_value,ec_value,toc_value,총질소_값,총인_값,클로로필-a_값,Day sin,Day cos,Year sin,Year cos
0,2.39329,6.84326,8.44667,106.64167,NaN,1.78600,0.00975,NaN,-8.039063e-12,1.000000,0.009848,0.999952
1,2.30697,6.84504,8.46367,106.18333,NaN,1.79475,0.00950,NaN,2.588190e-01,0.965926,0.010565,0.999944
2,2.27153,6.82695,8.45063,106.67500,NaN,1.75800,0.00933,NaN,5.000000e-01,0.866025,0.011282,0.999936
3,2.32228,6.84872,8.47916,106.00000,NaN,1.93725,0.00900,NaN,7.071068e-01,0.707107,0.011999,0.999928
4,2.31853,6.84714,8.47436,106.00000,NaN,1.91900,0.00975,NaN,8.660254e-01,0.500000,0.012715,0.999919
...,...,...,...,...,...,...,...,...,...,...,...,...
66451,22.86667,7.43417,8.96083,113.83333,1.89167,1.84300,0.01300,7.87500,-9.659258e-01,0.258819,-0.494157,-0.869373
66452,22.77500,7.32667,8.66750,114.00000,1.89500,1.85108,0.01300,7.09167,-8.660254e-01,0.500000,-0.494780,-0.869018
66453,22.63333,7.23417,8.40917,114.00000,1.85167,1.84700,0.01225,7.37500,-7.071068e-01,0.707107,-0.495403,-0.868664
66454,22.59167,7.21333,8.31750,114.00000,1.82667,1.89183,0.01200,8.73333,-5.000000e-01,0.866025,-0.496025,-0.868308


**낙동1**

In [4]:
file_names = [
              ['자동_해평_2016.xlsx',
               '자동_해평_2017.xlsx',
               '자동_해평_2018.xlsx',
               '자동_해평_2019.xlsx'],
              ['자동_도개_2016.xlsx',
               '자동_도개_2017.xlsx',
               '자동_도개_2018.xlsx',
               '자동_도개_2019.xlsx'],
              ['자동_신암_2016.xlsx',
               '자동_신암_2017.xlsx',
               '자동_신암_2018.xlsx',
               '자동_신암_2019.xlsx'],
             ]
gan_files = [ '자동_해평_gan.xlsx', '자동_도개_gan.xlsx', '자동_신암_gan.xlsx']
ori_files = [ '자동_해평_org.xlsx', '자동_도개_org.xlsx', '자동_신암_org.xlsx']
folder = 'data'

day = 24*60*60
year = (365.2425)*day

df_full = []
df = []

for loc in range(len(file_names)):
    
    df_loc = []
    for y in range(len(file_names[loc])):
        path = os.path.join(folder, file_names[loc][y])
        print(file_names[loc][y])
        df_loc.append(pd.read_excel(path))
    df_full.append(pd.concat(df_loc))
    if loc == 0:
        df.append(df_full[loc].iloc[:, 2:9].copy())
    else:
        df.append(df_full[loc].iloc[:, [2,3,4,5,6,7,10]].copy())
    date_time = pd.to_datetime(df_full[loc].iloc[:, 0], format='%Y.%m.%d %H:%M', utc=True)
    timestamp_s = date_time.map(datetime.datetime.timestamp)
    df[loc]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
    df[loc]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
    df[loc]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
    df[loc]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
    df[loc] = df[loc].reset_index(drop=True)


자동_해평_2016.xlsx
자동_해평_2017.xlsx
자동_해평_2018.xlsx
자동_해평_2019.xlsx
자동_도개_2016.xlsx
자동_도개_2017.xlsx
자동_도개_2018.xlsx
자동_도개_2019.xlsx
자동_신암_2016.xlsx
자동_신암_2017.xlsx
자동_신암_2018.xlsx
자동_신암_2019.xlsx


**hour df 추출**

In [195]:
file_names =  ['도개_2016.xlsx',
               '도개_2017.xlsx',
               '도개_2018.xlsx',
               '도개_2019.xlsx']

#file_name = file_names[0]

folder = 'water_data/낙동강1/자동'

date_dfs = []
for i in range(len(file_names)):
    path = os.path.join(folder, file_names[i])
    date_dfs.append(pd.read_excel(path, index_col='ymdh'))
    date_dfs[i] = pd.DataFrame(date_dfs[i], columns=[])

date_df = pd.concat(date_dfs)

In [190]:
file_names =  [
               #'도개_2015.xlsx',
               '도개_2016.xlsx',
               '도개_2017.xlsx',
               '도개_2018.xlsx',
               '도개_2019.xlsx',
               #'도개_2020.xlsx'
]

#file_name = file_names[0]

folder = 'water_data/nak/자동'

date_dfs = []
for i in range(len(file_names)):
    path = os.path.join(folder, file_names[i])
    date_dfs.append(pd.read_excel(path, index_col='ymdh'))
    date_dfs[i] = pd.DataFrame(date_dfs[i], columns=[])

date_df = pd.concat(date_dfs)

In [200]:
file_names =  [
               '도개_2013.xlsx',
               '도개_2014.xlsx',
               '도개_2015.xlsx',
               '도개_2016.xlsx',
               '도개_2017.xlsx',
               '도개_2018.xlsx',
               '도개_2019.xlsx',
               '도개_2020.xlsx'
]

#file_name = file_names[0]

folder = 'water_data/nak/자동'

date_dfs = []
for i in range(len(file_names)):
    path = os.path.join(folder, file_names[i])
    date_dfs.append(pd.read_excel(path, index_col='ymdh'))
    date_dfs[i] = pd.DataFrame(date_dfs[i], columns=[])

date_df = pd.concat(date_dfs)

In [201]:
date_df

""
ymdh
2013-01-01 00:00
2013-01-01 01:00
2013-01-01 02:00
2013-01-01 03:00
2013-01-01 04:00
...
2020-11-15 19:00
2020-11-15 20:00
2020-11-15 21:00


In [202]:
folder = 'data'
path = os.path.join(folder, 'date-13-20.xlsx')
date_df.to_excel(path)

In [198]:
tt_df = pd.read_excel(path, index_col='ymdh')

In [199]:
tt_df

""
ymdh
2016-01-01 00:00
2016-01-01 01:00
2016-01-01 02:00
2016-01-01 03:00
2016-01-01 04:00
...
2019-12-31 19:00
2019-12-31 20:00
2019-12-31 21:00


In [97]:
file_name = '낙단보_2016.xlsx'
folder = 'water_data/nak/조류'

In [98]:
path = os.path.join(folder, file_name)
tt_df = pd.read_excel(path)
tt_df.columns

Index(['dt', 'tme', 'stn_name', 'swmn_code', 'la', 'lo', 'rch_id', 'rch_did',
       'rch_sn', 'rch_div', 'geo_trib', 'inode_id', 'inode_did', 'onode_id',
       'onode_did', 'lu_rch_id', 'lu_rch_did', 'ru_rch_id', 'ru_rch_did',
       'di_rch_id', 'di_rch_did', 'ld_rch_id', 'ld_rch_did', 'rd_rch_id',
       'rd_rch_did', 'cat_id', 'cat_did', 'chck_de', 'nh3n', 'no3n', 'ph',
       'tur', 'cod', 'ec', 'bod', 'ss', 'do', 'tn', 'tp', 'wtp', 'sd',
       'chlorophylla', 'po4p', 'dtn', 'dtp', 'toc', 'gesm', 'mib', 'rainfall',
       'miclr', 'bluegac', 'nh3n_flag', 'no3n_flag', 'ph_flag', 'tur_flag',
       'cod_flag', 'ec_flag', 'bod_flag', 'ss_flag', 'do_flag', 'tn_flag',
       'tp_flag', 'wtp_flag', 'sd_flag', 'chlorophylla_flag', 'po4p_flag',
       'dtn_flag', 'dtp_flag', 'toc_flag', 'gesm_flag', 'mib_flag',
       'rainfall_flag', 'miclr_flag', 'bluegac_flag'],
      dtype='object')

In [100]:
tt_df['chlorophylla']

0      3.1
1      3.8
2      5.4
3      NaN
4      3.1
      ... 
843    9.4
844    3.1
845    3.6
846    4.6
847    9.4
Name: chlorophylla, Length: 848, dtype: float64

**총량측정망**

In [74]:
file_names = [
              ['말지천_2015.xlsx',
               '말지천_2016.xlsx',
               '말지천_2017.xlsx',
               '말지천_2018.xlsx',
               '말지천_2019.xlsx',
               '말지천_2020.xlsx'],
              ['병성천-1_2015.xlsx',
               '병성천-1_2016.xlsx',
               '병성천-1_2017.xlsx',
               '병성천-1_2018.xlsx',
               '병성천-1_2019.xlsx',
               '병성천-1_2020.xlsx'],
              ['상주3_2015.xlsx',
               '상주3_2016.xlsx',
               '상주3_2017.xlsx',
               '상주3_2018.xlsx',
               '상주3_2019.xlsx',
               '상주3_2020.xlsx'],
              ['위천6_2015.xlsx',
               '위천6_2016.xlsx',
               '위천6_2017.xlsx',
               '위천6_2018.xlsx',
               '위천6_2019.xlsx',
               '위천6_2020.xlsx'],
             ]
folder = 'water_data/nak/총량'

# what's nh3n, no3n, ss, po4p, dtn, dtp?
#qm_cols = ['nh3n', 'no3n', 'ph', 'cod', 'ec', 'bod', 'ss', 'do', 'tn', 'tp', 'chlorophylla', 'toc']
tm_cols = ['iem_flux', 'iem_wtrtp',
       'iem_ph', 'iem_doc', 'iem_bod', 'iem_cod', 'iem_ss', 'iem_ec', 'iem_tn',
       'iem_tp', 
       'iem_chla', 'iem_toc']
#df = df.loc[:, qm_cols]

tm_dfs = []
tm_inter_dfs = []
for j in range(len(file_names)):
    dfs = []
    for i in range(len(file_names[j])):
        path = os.path.join(folder, file_names[j][i])
        dfs.append(pd.read_excel(path, index_col='dt'))
        dfs[i] = dfs[i].loc[:, tm_cols]
    tm_dfs.append(pd.concat(dfs))
    tm_dfs[j] = pd.concat([date_df, tm_dfs[j]], axis=1)
    tm_dfs[j].reset_index(drop=True, inplace=True)
    first_series = tm_dfs[j].apply(pd.Series.first_valid_index)
    last_series = tm_dfs[j].apply(pd.Series.first_valid_index)
    for col in first_series.index:
        tm_dfs[j].loc[0, col] = tm_dfs[j].loc[first_series[col], col]
        tm_dfs[j].loc[len(tm_dfs[j])-1, col] = tm_dfs[j].loc[last_series[col], col]
    
    tm_dfs[j] = tm_dfs[j].interpolate(method='pchip')

In [76]:
tm_mean = []
tm_std = []

for i in range(len(tm_dfs)):
    tm_mean.append(tm_dfs[i].mean())
    tm_std.append(tm_dfs[i].std())
    tm_dfs[i] = (tm_dfs[i]-tm_mean[i])/tm_std[i]
    tm_dfs[i].columns=tm_dfs[i].columns+'%d'%(i+1)

In [77]:
tm_dfs[0]

,iem_flux1,iem_wtrtp1,iem_ph1,iem_doc1,iem_bod1,iem_cod1,iem_ss1,iem_ec1,iem_tn1,iem_tp1,iem_chla1,iem_toc1
0,-0.306699,-1.481488,-0.978770,1.426184,-0.790539,-1.134077,-0.204166,0.043708,0.295741,-0.721573,-0.064259,-1.048564
1,-0.306699,-1.481488,-0.978770,1.426184,-0.790539,-1.134077,-0.204166,0.043708,0.295741,-0.721573,-0.064259,-1.048564
2,-0.306699,-1.481488,-0.978770,1.426184,-0.790539,-1.134077,-0.204166,0.043708,0.295741,-0.721573,-0.064259,-1.048564
3,-0.306699,-1.481488,-0.978770,1.426184,-0.790539,-1.134077,-0.204166,0.043708,0.295741,-0.721573,-0.064259,-1.048564
4,-0.306699,-1.481488,-0.978770,1.426184,-0.790539,-1.134077,-0.204166,0.043708,0.295741,-0.721573,-0.064259,-1.048564
...,...,...,...,...,...,...,...,...,...,...,...,...
51499,-0.306698,-1.474994,-0.978747,1.423913,-0.790537,-1.131244,-0.203428,0.037661,0.293983,-0.718779,-0.064321,-1.045873
51500,-0.306699,-1.476618,-0.978757,1.424485,-0.790538,-1.131953,-0.203612,0.039171,0.294421,-0.719478,-0.064306,-1.046546
51501,-0.306699,-1.478241,-0.978765,1.425054,-0.790539,-1.132661,-0.203796,0.040682,0.294861,-0.720177,-0.064290,-1.047219
51502,-0.306699,-1.479864,-0.978769,1.425621,-0.790539,-1.133369,-0.203981,0.042195,0.295301,-0.720875,-0.064275,-1.047892


In [10]:
file_names = [
              ['상주1_2016.xlsx',
               '상주1_2017.xlsx',
               '상주1_2018.xlsx',
               '상주1_2019.xlsx'],
              ['영강2-1_2016.xlsx',
               '영강2-1_2017.xlsx',
               '영강2-1_2018.xlsx',
               '영강2-1_2019.xlsx'],
             ]
folder = 'water_data/낙동강2/수질측정망'

file_names = [
              ['병성천-1_2015.xlsx',
               '병성천-1_2016.xlsx',
               '병성천-1_2017.xlsx',
               '병성천-1_2018.xlsx',
               '병성천-1_2019.xlsx',
               '병성천-1_2020.xlsx'],
              ['상주2_2015.xlsx',
               '상주2_2016.xlsx',
               '상주2_2017.xlsx',
               '상주2_2018.xlsx',
               '상주2_2019.xlsx',
               '상주2_2020.xlsx'],
              ['상주3_2015.xlsx',
               '상주3_2016.xlsx',
               '상주3_2017.xlsx',
               '상주3_2018.xlsx',
               '상주3_2019.xlsx',
               '상주3_2020.xlsx'],
             ]
folder = 'water_data/nak/수질'

# what's nh3n, no3n, ss, po4p, dtn, dtp?
qm_cols = ['nh3n', 'no3n', 'ph', 'cod', 'ec', 'bod', 'ss', 'do', 'tn', 'tp', 'chlorophylla', 'toc']
#df = df.loc[:, qm_cols]

qm_dfs = []
qm_inter_dfs = []
for j in range(len(file_names)):
    dfs = []
    for i in range(len(file_names[j])):
        path = os.path.join(folder, file_names[j][i])
        dfs.append(pd.read_excel(path, index_col='dt'))
        dfs[i] = dfs[i].loc[:, qm_cols]
    qm_dfs.append(pd.concat(dfs))
    qm_dfs[j] = pd.concat([date_df, qm_dfs[j]], axis=1)
    qm_dfs[j].reset_index(drop=True, inplace=True)
    first_series = qm_dfs[j].apply(pd.Series.first_valid_index)
    last_series = qm_dfs[j].apply(pd.Series.first_valid_index)
    for col in first_series.index:
        qm_dfs[j].loc[0, col] = qm_dfs[j].loc[first_series[col], col]
        qm_dfs[j].loc[len(qm_dfs[j])-1, col] = qm_dfs[j].loc[last_series[col], col]
    
    qm_dfs[j] = qm_dfs[j].interpolate(method='pchip')
    

In [14]:
file_name = '말지천_2015.xlsx'
folder = 'water_data/nak/총량'

path = os.path.join(folder, file_name)
tt_df = pd.read_excel(path)
tt_df.columns

Index(['dt', 'tme', 'stn_name', 'wqmn_code', 'la', 'lo', 'rch_id', 'rch_did',
       'rch_sn', 'rch_div', 'geo_trib', 'inode_id', 'inode_did', 'onode_id',
       'onode_did', 'lu_rch_id', 'lu_rch_did', 'ru_rch_id', 'ru_rch_did',
       'di_rch_id', 'di_rch_did', 'ld_rch_id', 'ld_rch_did', 'rd_rch_id',
       'rd_rch_did', 'cat_id', 'cat_did', 'chck_de', 'iem_flux', 'iem_wtrtp',
       'iem_ph', 'iem_doc', 'iem_bod', 'iem_cod', 'iem_ss', 'iem_ec', 'iem_tn',
       'iem_dtn', 'iem_no3n', 'iem_nh4n', 'iem_tp', 'iem_dtp', 'iem_op4',
       'iem_chla', 'iem_toc', 'iem_flux_flag', 'iem_wtrtp_flag', 'iem_ph_flag',
       'iem_doc_flag', 'iem_bod_flag', 'iem_cod_flag', 'iem_ss_flag',
       'iem_ec_flag', 'iem_tn_flag', 'iem_dtn_flag', 'iem_no3n_flag',
       'iem_nh4n_flag', 'iem_tp_flag', 'iem_dtp_flag', 'iem_op4_flag',
       'iem_chla_flag', 'iem_toc_flag'],
      dtype='object')

In [ ]:
folder = 'water_data/nak/총량'

In [45]:
file_names = [
              ['병성천-1_2015.xlsx',
               '병성천-1_2016.xlsx',
               '병성천-1_2017.xlsx',
               '병성천-1_2018.xlsx',
               '병성천-1_2019.xlsx',
               '병성천-1_2020.xlsx'],
              ['상주2_2015.xlsx',
               '상주2_2016.xlsx',
               '상주2_2017.xlsx',
               '상주2_2018.xlsx',
               '상주2_2019.xlsx',
               '상주2_2020.xlsx'],
              ['상주3_2015.xlsx',
               '상주3_2016.xlsx',
               '상주3_2017.xlsx',
               '상주3_2018.xlsx',
               '상주3_2019.xlsx',
               '상주3_2020.xlsx'],
             ]
folder = 'water_data/nak/수질'

# what's nh3n, no3n, ss, po4p, dtn, dtp?
qm_cols = ['nh3n', 'no3n', 'ph', 'cod', 'ec', 'bod', 'ss', 'do', 'tn', 'tp', 'chlorophylla', 'toc']
#df = df.loc[:, qm_cols]

qm_dfs = []
qm_inter_dfs = []
for j in range(len(file_names)):
    dfs = []
    for i in range(len(file_names[j])):
        path = os.path.join(folder, file_names[j][i])
        dfs.append(pd.read_excel(path, index_col='dt'))
        dfs[i] = dfs[i].loc[:, qm_cols]
    qm_dfs.append(pd.concat(dfs))
    qm_dfs[j] = pd.concat([date_df, qm_dfs[j]], axis=1)
    qm_dfs[j].reset_index(drop=True, inplace=True)
    first_series = qm_dfs[j].apply(pd.Series.first_valid_index)
    last_series = qm_dfs[j].apply(pd.Series.first_valid_index)
    for col in first_series.index:
        qm_dfs[j].loc[0, col] = qm_dfs[j].loc[first_series[col], col]
        qm_dfs[j].loc[len(qm_dfs[j])-1, col] = qm_dfs[j].loc[last_series[col], col]
    
    qm_dfs[j] = qm_dfs[j].interpolate(method='pchip')

In [90]:
a = qm_dfs[0].loc[:, 'chlorophylla'].to_numpy()
plt.figure()
#plt.scatter(qm_dfs[0].loc[:, 'chlorophylla'].index, qm_dfs[0].loc[:, 'chlorophylla'])
plt.plot(qm_dfs[0].loc[:, 'chlorophylla'])
#plt.plot(qm_inter_dfs[0].loc[:, 'chlorophylla'])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Normalize and rename columns**

In [15]:
qm_mean = []
qm_std = []

for i in range(len(qm_dfs)):
    qm_mean.append(qm_dfs[i].mean())
    qm_std.append(qm_dfs[i].std())
    qm_dfs[i] = (qm_dfs[i]-qm_mean[i])/qm_std[i]
    qm_dfs[i].columns=qm_dfs[i].columns+'%d'%(i+1)

In [16]:
qm_dfs[1]

,nh3n2,no3n2,ph2,cod2,ec2,bod2,ss2,do2,tn2,tp2,chlorophylla2,toc2
0,1.371398,0.830867,-0.743391,-1.034479,0.054743,-1.087629,-0.456538,1.660066,0.525929,-0.861533,-1.504608,-1.296197
1,1.371398,0.830867,-0.743391,-1.034479,0.054743,-1.087629,-0.456538,1.660066,0.525929,-0.861533,-1.504608,-1.296197
2,1.371398,0.830867,-0.743391,-1.034479,0.054743,-1.087629,-0.456538,1.660066,0.525929,-0.861533,-1.504608,-1.296197
3,1.371398,0.830867,-0.743391,-1.034479,0.054743,-1.087629,-0.456538,1.660066,0.525929,-0.861533,-1.504608,-1.296197
4,1.371398,0.830867,-0.743391,-1.034479,0.054743,-1.087629,-0.456538,1.660066,0.525929,-0.861533,-1.504608,-1.296197
...,...,...,...,...,...,...,...,...,...,...,...,...
51499,1.371395,0.822052,-0.737147,-1.014200,0.050997,-1.086018,-0.439645,1.643192,0.520192,-0.840716,-1.497470,-1.284500
51500,1.371396,0.824250,-0.738704,-1.019257,0.051935,-1.086420,-0.443858,1.647406,0.521623,-0.845908,-1.499250,-1.287422
51501,1.371397,0.826452,-0.740264,-1.024323,0.052871,-1.086822,-0.448078,1.651623,0.523057,-0.851108,-1.501033,-1.290346
51502,1.371397,0.828658,-0.741826,-1.029397,0.053807,-1.087226,-0.452305,1.655843,0.524492,-0.856316,-1.502819,-1.293271


**낙동강2 자동측정망**

In [158]:
file_names = [
              ['낙동강1/자동/도개_2016.xlsx',
               '낙동강1/자동/도개_2017.xlsx',
               '낙동강1/자동/도개_2018.xlsx',
               '낙동강1/자동/도개_2019.xlsx'],
              ['낙동강1/자동/신암_2016.xlsx',
               '낙동강1/자동/신암_2017.xlsx',
               '낙동강1/자동/신암_2018.xlsx',
               '낙동강1/자동/신암_2019.xlsx'],
              ['낙동강1/자동/회상_2016.xlsx',
               '낙동강1/자동/회상_2017.xlsx',
               '낙동강1/자동/회상_2018.xlsx',
               '낙동강1/자동/회상_2019.xlsx'],
#              ['낙동강2/자동측정망/풍양_2016.xlsx',
#               '낙동강2/자동측정망/풍양_2017.xlsx',
#               '낙동강2/자동측정망/풍양_2018.xlsx',
#               '낙동강2/자동측정망/풍양_2019.xlsx'],
             ]
#gan_files = [ '도개_gan.xlsx', '신암_gan.xlsx', '회상_gan.xlsx', '풍양_gan.xlsx']
#ori_files = [ '도개_org.xlsx', '신암_org.xlsx', '회상_org.xlsx', '풍양_gan.xlsx']
gan_files = [ '도개_gan.xlsx', '신암_gan.xlsx', '회상_gan.xlsx']
ori_files = [ '도개_org.xlsx', '신암_org.xlsx', '회상_org.xlsx']
folder = 'water_data'

day = 24*60*60
year = (365.2425)*day

df_full = []
df = []

auto_cols = ['tmpr_value', 'ph_value', 'do_value', 'ec_value', 'toc_value', '총질소_값', '총인_값', '클로로필-a_값']

for loc in range(len(file_names)):
    
    df_loc = []
    for y in range(len(file_names[loc])):
        path = os.path.join(folder, file_names[loc][y])
        print(file_names[loc][y])
        df_loc.append(pd.read_excel(path))
    df_full.append(pd.concat(df_loc))
    df.append(df_full[loc].loc[:, auto_cols].copy())
    date_time = pd.to_datetime(df_full[loc].loc[:, 'ymdh'], format='%Y.%m.%d %H:%M', utc=True)
    timestamp_s = date_time.map(datetime.datetime.timestamp)
    df[loc]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
    df[loc]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
    df[loc]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
    df[loc]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
    df[loc] = df[loc].reset_index(drop=True)

낙동강1/자동/도개_2016.xlsx
낙동강1/자동/도개_2017.xlsx
낙동강1/자동/도개_2018.xlsx
낙동강1/자동/도개_2019.xlsx
낙동강1/자동/신암_2016.xlsx
낙동강1/자동/신암_2017.xlsx
낙동강1/자동/신암_2018.xlsx
낙동강1/자동/신암_2019.xlsx
낙동강1/자동/회상_2016.xlsx
낙동강1/자동/회상_2017.xlsx
낙동강1/자동/회상_2018.xlsx
낙동강1/자동/회상_2019.xlsx
낙동강2/자동측정망/풍양_2016.xlsx
낙동강2/자동측정망/풍양_2017.xlsx
낙동강2/자동측정망/풍양_2018.xlsx
낙동강2/자동측정망/풍양_2019.xlsx


In [5]:
file_names = [
              ['도개_2013.xlsx',
               '도개_2014.xlsx',
               '도개_2015.xlsx',
               '도개_2016.xlsx',
               '도개_2017.xlsx',
               '도개_2018.xlsx',
               '도개_2019.xlsx',
               '도개_2020.xlsx'],
              ['신암_2013.xlsx',
               '신암_2014.xlsx',
               '신암_2015.xlsx',
               '신암_2016.xlsx',
               '신암_2017.xlsx',
               '신암_2018.xlsx',
               '신암_2019.xlsx',
               '신암_2020.xlsx'],
#              ['낙동강2/자동측정망/풍양_2016.xlsx',
#               '낙동강2/자동측정망/풍양_2017.xlsx',
#               '낙동강2/자동측정망/풍양_2018.xlsx',
#               '낙동강2/자동측정망/풍양_2019.xlsx'],
             ]
#gan_files = [ '도개_gan.xlsx', '신암_gan.xlsx', '회상_gan.xlsx', '풍양_gan.xlsx']
#ori_files = [ '도개_org.xlsx', '신암_org.xlsx', '회상_org.xlsx', '풍양_gan.xlsx']
gan_files = [ '도개4_gan.xlsx', '신암4_gan.xlsx']
ori_files = [ '도개4_org.xlsx', '신암4_org.xlsx']
folder = 'water_data/nak/자동'

day = 24*60*60
year = (365.2425)*day

df_full = []
df = []

auto_cols = ['tmpr_value', 'ph_value', 'do_value', 'ec_value', 'toc_value', '총질소_값', '총인_값', '클로로필-a_값']

for loc in range(len(file_names)):
    
    df_loc = []
    for y in range(len(file_names[loc])):
        path = os.path.join(folder, file_names[loc][y])
        print(file_names[loc][y])
        df_loc.append(pd.read_excel(path))
    df_full.append(pd.concat(df_loc))
    df.append(df_full[loc].loc[:, auto_cols].copy())
    date_time = pd.to_datetime(df_full[loc].loc[:, 'ymdh'], format='%Y.%m.%d %H:%M', utc=True)
    timestamp_s = date_time.map(datetime.datetime.timestamp)
    df[loc]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
    df[loc]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
    df[loc]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
    df[loc]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
    df[loc] = df[loc].reset_index(drop=True)

도개_2013.xlsx
도개_2014.xlsx
도개_2015.xlsx
도개_2016.xlsx
도개_2017.xlsx
도개_2018.xlsx
도개_2019.xlsx
도개_2020.xlsx
신암_2013.xlsx
신암_2014.xlsx
신암_2015.xlsx
신암_2016.xlsx
신암_2017.xlsx
신암_2018.xlsx
신암_2019.xlsx
신암_2020.xlsx


In [89]:
file_names = [
              ['도개_2015.xlsx',
               '도개_2016.xlsx',
               '도개_2017.xlsx',
               '도개_2018.xlsx',
               '도개_2019.xlsx',
               '도개_2020.xlsx'],
              ['신암_2015.xlsx',
               '신암_2016.xlsx',
               '신암_2017.xlsx',
               '신암_2018.xlsx',
               '신암_2019.xlsx',
               '신암_2020.xlsx'],
#              ['낙동강2/자동측정망/풍양_2016.xlsx',
#               '낙동강2/자동측정망/풍양_2017.xlsx',
#               '낙동강2/자동측정망/풍양_2018.xlsx',
#               '낙동강2/자동측정망/풍양_2019.xlsx'],
             ]
#gan_files = [ '도개_gan.xlsx', '신암_gan.xlsx', '회상_gan.xlsx', '풍양_gan.xlsx']
#ori_files = [ '도개_org.xlsx', '신암_org.xlsx', '회상_org.xlsx', '풍양_gan.xlsx']
gan_files = [ '도개3_gan.xlsx', '신암3_gan.xlsx']
ori_files = [ '도개3_org.xlsx', '신암3_org.xlsx']
folder = 'water_data/nak/자동'

day = 24*60*60
year = (365.2425)*day

df_full = []
df = []

auto_cols = ['tmpr_value', 'ph_value', 'do_value', 'ec_value', 'toc_value', '총질소_값', '총인_값', '클로로필-a_값']

for loc in range(len(file_names)):
    
    df_loc = []
    for y in range(len(file_names[loc])):
        path = os.path.join(folder, file_names[loc][y])
        print(file_names[loc][y])
        df_loc.append(pd.read_excel(path))
    df_full.append(pd.concat(df_loc))
    df.append(df_full[loc].loc[:, auto_cols].copy())
    date_time = pd.to_datetime(df_full[loc].loc[:, 'ymdh'], format='%Y.%m.%d %H:%M', utc=True)
    timestamp_s = date_time.map(datetime.datetime.timestamp)
    df[loc]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
    df[loc]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
    df[loc]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
    df[loc]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
    df[loc] = df[loc].reset_index(drop=True)

도개_2015.xlsx
도개_2016.xlsx
도개_2017.xlsx
도개_2018.xlsx
도개_2019.xlsx
도개_2020.xlsx
신암_2015.xlsx
신암_2016.xlsx
신암_2017.xlsx
신암_2018.xlsx
신암_2019.xlsx
신암_2020.xlsx


In [38]:
file_names = [
              ['현도_2013.xlsx',
               '현도_2014.xlsx',
               '현도_2015.xlsx',
               '현도_2016.xlsx',
               '현도_2017.xlsx',
               '현도_2018.xlsx',
               '현도_2019.xlsx',
               '현도_2020.xlsx'],
              ['대청호_2013.xlsx',
               '대청호_2014.xlsx',
               '대청호_2015.xlsx',
               '대청호_2016.xlsx',
               '대청호_2017.xlsx',
               '대청호_2018.xlsx',
               '대청호_2019.xlsx',
               '대청호_2020.xlsx'],
              ['장계_2013.xlsx',
               '장계_2014.xlsx',
               '장계_2015.xlsx',
               '장계_2016.xlsx',
               '장계_2017.xlsx',
               '장계_2018.xlsx',
               '장계_2019.xlsx',
               '장계_2020.xlsx'],
#              ['낙동강2/자동측정망/풍양_2016.xlsx',
#               '낙동강2/자동측정망/풍양_2017.xlsx',
#               '낙동강2/자동측정망/풍양_2018.xlsx',
#               '낙동강2/자동측정망/풍양_2019.xlsx'],
             ]
#gan_files = [ '도개_gan.xlsx', '신암_gan.xlsx', '회상_gan.xlsx', '풍양_gan.xlsx']
#ori_files = [ '도개_org.xlsx', '신암_org.xlsx', '회상_org.xlsx', '풍양_gan.xlsx']
gan_files = [ '현도2_gan.xlsx', '대청호2_gan.xlsx', '장계2_gan.xlsx']
ori_files = [ '현도2_ori.xlsx', '대청호2_ori.xlsx', '장계2_ori.xlsx']
folder = 'water_data/geum/자동'

day = 24*60*60
year = (365.2425)*day

df_full = []
df = []

auto_cols = ['tmpr_value', 'ph_value', 'do_value', 'ec_value', 'toc_value', '총질소_값', '총인_값', '클로로필-a_값']

for loc in range(len(file_names)):
    
    df_loc = []
    for y in range(len(file_names[loc])):
        path = os.path.join(folder, file_names[loc][y])
        print(file_names[loc][y])
        df_loc.append(pd.read_excel(path))
    df_full.append(pd.concat(df_loc))
    df.append(df_full[loc].loc[:, auto_cols].copy())
    date_time = pd.to_datetime(df_full[loc].loc[:, 'ymdh'], format='%Y.%m.%d %H:%M', utc=True)
    timestamp_s = date_time.map(datetime.datetime.timestamp)
    df[loc]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
    df[loc]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
    df[loc]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
    df[loc]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
    df[loc] = df[loc].reset_index(drop=True)

현도_2013.xlsx
현도_2014.xlsx
현도_2015.xlsx
현도_2016.xlsx
현도_2017.xlsx
현도_2018.xlsx
현도_2019.xlsx
현도_2020.xlsx
대청호_2013.xlsx
대청호_2014.xlsx
대청호_2015.xlsx
대청호_2016.xlsx
대청호_2017.xlsx
대청호_2018.xlsx
대청호_2019.xlsx
대청호_2020.xlsx
장계_2013.xlsx
장계_2014.xlsx
장계_2015.xlsx
장계_2016.xlsx
장계_2017.xlsx
장계_2018.xlsx
장계_2019.xlsx
장계_2020.xlsx


In [102]:
file_names = [
              ['현도_2015.xlsx',
               '현도_2016.xlsx',
               '현도_2017.xlsx',
               '현도_2018.xlsx',
               '현도_2019.xlsx',
               '현도_2020.xlsx'],
              ['대청호_2015.xlsx',
               '대청호_2016.xlsx',
               '대청호_2017.xlsx',
               '대청호_2018.xlsx',
               '대청호_2019.xlsx',
               '대청호_2020.xlsx'],
              ['장계_2015.xlsx',
               '장계_2016.xlsx',
               '장계_2017.xlsx',
               '장계_2018.xlsx',
               '장계_2019.xlsx',
               '장계_2020.xlsx'],
#              ['낙동강2/자동측정망/풍양_2016.xlsx',
#               '낙동강2/자동측정망/풍양_2017.xlsx',
#               '낙동강2/자동측정망/풍양_2018.xlsx',
#               '낙동강2/자동측정망/풍양_2019.xlsx'],
             ]
#gan_files = [ '도개_gan.xlsx', '신암_gan.xlsx', '회상_gan.xlsx', '풍양_gan.xlsx']
#ori_files = [ '도개_org.xlsx', '신암_org.xlsx', '회상_org.xlsx', '풍양_gan.xlsx']
gan_files = [ '현도_gan.xlsx', '대청호_gan.xlsx', '장계_gan.xlsx']
ori_files = [ '현도_ori.xlsx', '대청호_ori.xlsx', '장계_ori.xlsx']
folder = 'water_data/geum/자동'

day = 24*60*60
year = (365.2425)*day

df_full = []
df = []

auto_cols = ['tmpr_value', 'ph_value', 'do_value', 'ec_value', 'toc_value', '총질소_값', '총인_값', '클로로필-a_값']

for loc in range(len(file_names)):
    
    df_loc = []
    for y in range(len(file_names[loc])):
        path = os.path.join(folder, file_names[loc][y])
        print(file_names[loc][y])
        df_loc.append(pd.read_excel(path))
    df_full.append(pd.concat(df_loc))
    df.append(df_full[loc].loc[:, auto_cols].copy())
    date_time = pd.to_datetime(df_full[loc].loc[:, 'ymdh'], format='%Y.%m.%d %H:%M', utc=True)
    timestamp_s = date_time.map(datetime.datetime.timestamp)
    df[loc]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
    df[loc]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
    df[loc]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
    df[loc]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
    df[loc] = df[loc].reset_index(drop=True)

현도_2015.xlsx
현도_2016.xlsx
현도_2017.xlsx
현도_2018.xlsx
현도_2019.xlsx
현도_2020.xlsx
대청호_2015.xlsx
대청호_2016.xlsx
대청호_2017.xlsx
대청호_2018.xlsx
대청호_2019.xlsx
대청호_2020.xlsx
장계_2015.xlsx
장계_2016.xlsx
장계_2017.xlsx
장계_2018.xlsx
장계_2019.xlsx
장계_2020.xlsx


In [165]:
file_names = [
              ['나주_2013.xlsx',
               '나주_2014.xlsx',
               '나주_2015.xlsx',
               '나주_2016.xlsx',
               '나주_2017.xlsx',
               '나주_2018.xlsx',
               '나주_2019.xlsx',
               '나주_2020.xlsx'],
              ['용봉_2013.xlsx',
               '용봉_2014.xlsx',
               '용봉_2015.xlsx',
               '용봉_2016.xlsx',
               '용봉_2017.xlsx',
               '용봉_2018.xlsx',
               '용봉_2019.xlsx',
               '용봉_2020.xlsx'],
              ['우치_2013.xlsx',
               '우치_2014.xlsx',
               '우치_2015.xlsx',
               '우치_2016.xlsx',
               '우치_2017.xlsx',
               '우치_2018.xlsx',
               '우치_2019.xlsx',
               '우치_2020.xlsx'],
#              ['낙동강2/자동측정망/풍양_2016.xlsx',
#               '낙동강2/자동측정망/풍양_2017.xlsx',
#               '낙동강2/자동측정망/풍양_2018.xlsx',
#               '낙동강2/자동측정망/풍양_2019.xlsx'],
             ]
#gan_files = [ '도개_gan.xlsx', '신암_gan.xlsx', '회상_gan.xlsx', '풍양_gan.xlsx']
#ori_files = [ '도개_org.xlsx', '신암_org.xlsx', '회상_org.xlsx', '풍양_gan.xlsx']
gan_files = [ '나주_gan.xlsx', '용봉_gan.xlsx', '우치_gan.xlsx']
ori_files = [ '나주_org.xlsx', '용봉_org.xlsx', '우치_org.xlsx']
folder = 'water_data/yeong/자동'

day = 24*60*60
year = (365.2425)*day

df_full = []
df = []

auto_cols = ['tmpr_value', 'ph_value', 'do_value', 'ec_value', 'toc_value', '총질소_값', '총인_값', '클로로필-a_값']

for loc in range(len(file_names)):
    
    df_loc = []
    for y in range(len(file_names[loc])):
        path = os.path.join(folder, file_names[loc][y])
        print(file_names[loc][y])
        df_loc.append(pd.read_excel(path))
    df_full.append(pd.concat(df_loc))
    df.append(df_full[loc].loc[:, auto_cols].copy())
    date_time = pd.to_datetime(df_full[loc].loc[:, 'ymdh'], format='%Y.%m.%d %H:%M', utc=True)
    timestamp_s = date_time.map(datetime.datetime.timestamp)
    df[loc]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
    df[loc]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
    df[loc]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
    df[loc]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
    df[loc] = df[loc].reset_index(drop=True)

나주_2013.xlsx
나주_2014.xlsx
나주_2015.xlsx
나주_2016.xlsx
나주_2017.xlsx
나주_2018.xlsx
나주_2019.xlsx
나주_2020.xlsx
용봉_2013.xlsx
용봉_2014.xlsx
용봉_2015.xlsx
용봉_2016.xlsx
용봉_2017.xlsx
용봉_2018.xlsx
용봉_2019.xlsx
용봉_2020.xlsx
우치_2013.xlsx
우치_2014.xlsx
우치_2015.xlsx
우치_2016.xlsx
우치_2017.xlsx
우치_2018.xlsx
우치_2019.xlsx
우치_2020.xlsx


In [207]:
# normalize data

df_all = pd.concat(df)
df_all

train_mean = df_all.mean()
train_std = df_all.std()
for i in range(len(file_names)):
    df[i] = (df[i]-train_mean)/train_std

In [209]:
plt.figure()
plt.plot(df[0].iloc[:, 6])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [188]:
#df[0].iloc[-3750:-(3850-50)]
df[0].shape[0] - 65074

3734

In [189]:
3734//24*24

3720

**data map**

In [210]:
#총 location 수
n_loc = len(df)
# 그림 col 수
n_col = n_loc*2
plt.figure(figsize=(9,10))
isnan = np.isnan(df_all).astype(int)
#isnan = np.isnan(df[0]).astype(int)
data = isnan.to_numpy()
n = data.shape[0]
seq_len = n//n_col
for i in range(n_col):
    plt.subplot(101+(n_col*10)+i)
    plt.imshow(data[i*seq_len:(i+1)*seq_len, 0:8], aspect='auto')
    plt.yticks([])
plt.show()

/home/kotech/workspace/venv-tensor2n-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [211]:
train_df = df[0]
val_df = df[0]
test_df = df[0]

In [212]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
            #train_df=None, val_df=None, test_df=None,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [213]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [214]:
def plot(self, model=None, plot_col='T (degC)', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [215]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [216]:
def sample_batch_index(total, batch_size):
    '''Sample index of the mini-batch.

    Args:
        - total: total number of samples
        - batch_size: batch size

    Returns:
        - batch_idx: batch index
    '''
    total_idx = np.random.permutation(total)
    batch_idx = total_idx[:batch_size]
    return batch_idx

In [217]:
def binary_sampler(p, shape):
  '''Sample binary random variables.
  
  Args:
    - p: probability of 1
    - shape: matrix shape
    
  Returns:
    - binary_random_matrix: generated binary random matrix.
  '''
  unif_random_matrix = np.random.uniform(0., 1., size = shape)
  binary_random_matrix = 1*(unif_random_matrix < p)
  return binary_random_matrix

In [218]:
def uniform_sampler(low, high, shape):
  '''Sample uniform random variables.
  
  Args:
    - low: low limit
    - high: high limit
    - rows: the number of rows
    - cols: the number of columns
    
  Returns:
    - uniform_random_matrix: generated uniform random matrix.
  '''
  return np.random.uniform(low, high, size = shape)

In [219]:
def normalization (data, parameters=None):
  '''Normalize data in [0, 1] range.
  
  Args:
    - data: original data
  
  Returns:
    - norm_data: normalized data
    - norm_parameters: min_val, max_val for each feature for renormalization
  '''

  # Parameters
  _, dim = data.shape
  norm_data = data.copy()

  if parameters is None:

    # MixMax normalization
    min_val = np.zeros(dim)
    max_val = np.zeros(dim)
   
    # For each dimension
    for i in range(dim):
      min_val[i] = np.nanmin(norm_data[:,i])
      norm_data[:,i] = norm_data[:,i] - np.nanmin(norm_data[:,i])
      max_val[i] = np.nanmax(norm_data[:,i])
      norm_data[:,i] = norm_data[:,i] / (np.nanmax(norm_data[:,i]) + 1e-6)

    # Return norm_parameters for renormalization
    norm_parameters = {'min_val': min_val,
                       'max_val': max_val}
  else:
    min_val = parameters['min_val']
    max_val = parameters['max_val']

    # For each dimension
    for i in range(dim):
      norm_data[:,i] = norm_data[:,i] - min_val[i]
      norm_data[:,i] = norm_data[:,i] / (max_val[i] + 1e-6)

    norm_parameters = parameters

  return norm_data, norm_parameters

In [220]:
class MissData(object):
    def __init__(self, load_dir=None):
        if load_dir:
            self.missarr = np.load(os.path.join(load_dir, 'miss.npy'))
            self.idxarr = np.load(os.path.join(load_dir, 'idx.npy'))
            
    def make_missdata(self, data_x, missrate=0.2):
        data = data_x.copy()
        rows, cols = data_x.shape
        total_no = rows*cols
        total_miss_no = np.round(total_no*missrate).astype(int)
        total_idx = self.idxarr.shape[0]
        idxarr = self.idxarr
        missarr = self.missarr
        #print(total_miss_no)
        miss_no = 0
        cum_no = self.idxarr[:,3:4]
        cum_no = cum_no.reshape((total_idx))
        cum_sum = np.max(cum_no)
        #print(cum_no)
        #print(total_idx)
        while True:
            loc_count = np.around(np.random.random()*cum_sum)
            #print('loc_count =', loc_count)
            idx = len(cum_no[cum_no <= loc_count])-1
            #print(cum_no[cum_no <= loc_count])
            #print('idx =', idx)
            startnan = idxarr[idx][0]
            nanlen = idxarr[idx][2]
            loc = np.around(np.random.random()*(rows-nanlen)).astype(int)
            #print('loc =', loc)
            #print(loc_count, idx)
            #print(idxarr[idx])
            #data_copy = data[loc:loc+nanlen].copy()
            data_copy = data[loc:loc+nanlen]
            #print('startnan=', startnan)
            #isnan = missarr[startnan:startnan+nanlen].copy()
            isnan = missarr[startnan:startnan+nanlen]
            #print('isnan =',isnan)
            miss_no += idxarr[idx][1]
            if (miss_no > total_miss_no):
                break
            data_copy[isnan==1] = np.nan
            data[loc:loc+nanlen] = data_copy
        #print('miss_data =', data)
        return data
    
    def save(data, max_tseq, save_dir='save'):
        no, dim = data.shape
        #print((no, dim))
        isnan = np.isnan(data).astype(int)
        isany = np.any(isnan, axis=1).astype(int)
        shifted = np.roll(isany, 1)
        shifted[0] = 1
        #print(isnan)
        #print(isany.astype(int))
        #print(shifted)
        startnan = ((isany == 1) & (shifted ==0)).astype(int)
        #print(startnan)
        group = startnan.cumsum()
        group = group*isany
        #print(group)
        n = np.max(group)
        #print(n)
        missarr = None
        cum_no = 0
        rowidx = 0
        for i in range(1, n+1):
            g = (group == i).astype(int)
            i = np.argmax(g)
            rows = g.sum()
            #print(len)
            #print(i)
            #print(type(missarr))
            if rows <= max_tseq:
                nanseq = isnan[i:i+rows, :]
                no = np.sum(nanseq)
                #print(no)
                if missarr is None:
                    missarr = nanseq
                    idxarr = np.array([[rowidx, no, rows, cum_no]])
                else:
                    missarr = np.concatenate((missarr, nanseq))
                    idxarr = np.concatenate((idxarr, [[rowidx, no, rows, cum_no]]), axis=0)
                cum_no += no
                rowidx += rows

        #print(idxarr)
        miss_npy_file = os.path.join(save_dir, 'miss.npy')
        idx_npy_file = os.path.join(save_dir, 'idx.npy')
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        np.save(miss_npy_file, missarr)
        np.save(idx_npy_file, idxarr)
        print('miss_data file saved')

In [221]:
norm_df = pd.concat(df,axis=0)
norm_data = norm_df.to_numpy()
MissData.save(norm_data, max_tseq = 24)

miss_data file saved


In [222]:
def interpolate(np_data, max_gap=3):
    #n = np_data.shape[1]
    data = pd.DataFrame(np_data)
    #data[0][0] = np.nan
    #data[0][1] = np.nan
    #data[0][2] = np.nan
    #data[data.columns[0]][0] = np.nan
    #data[data.columns[0]][1] = np.nan
    #data[data.columns[0]][2] = np.nan
    
    # create mask
    mask = data.copy()
    grp = ((mask.notnull() != mask.shift().notnull()).cumsum())
    grp['ones'] = 1
    for i in data.columns:
        mask[i] = (grp.groupby(i)['ones'].transform('count') < max_gap) | data[i].notnull()
    data = data.interpolate(method='polynomial', order=5, limit=max_gap, axis=0).bfill()[mask]
    return data.to_numpy()
    #return data
    
#filled_data = interpolate(norm_data, max_gap=3)
#np.arange(0, 5, dtype=int)
#['%d'%val for val in range(0,5)]

In [223]:
class GainDataGenerator(keras.utils.Sequence):
    'Generates data for GAIN'
    def __init__(self,
                 data_list,
                 batch_size=32,
                 input_width=24*3,
                 label_width=24*3,
                 shift=0,
                 fill_no=4,
                 miss_rate=0.2,
                 hint_rate=0.9,
                 normalize=True,
                 miss_pattern=None,
                 alpha=100.):
        'Initialization'
        window_size = input_width
        
        # interpollation
        filled_data = []
        for data in data_list:
            data = interpolate(data, max_gap=fill_no)
            filled_data.append(data)
            
        data_list = filled_data
        
        # whole data
        self.data = np.concatenate(data_list)

        # TO-DO
        
        # pre calculation for  sequence data
        last_cum = 0
        cums = []
        for data in data_list:
            isnan = np.isnan(data)
            isany = np.any(isnan, axis=1)
            #shift same as pd.shift(isany, fill_value=True)
            shifted = np.roll(isany, 1)
            shifted[0] = True # set to nan
            
            start_seq = ((isany == False) & (shifted == True)).astype(int)
            cum = start_seq.cumsum()
            cum += last_cum
            last_cum = np.max(cum)
            cum[isany] = 0
            cums.append(cum)
            
        
        # normlize for spam
        if normalize:
            self.data, norm_param = normalization(self.data)
        #print(norm_param)
        
        # Define mask matrix
        if miss_pattern is None:
            self.data_m = binary_sampler(1-miss_rate, self.data.shape)
        else:
            #MissData.save(self.data, max_tseq = 12)
            self.miss = MissData(load_dir='save')
            self.miss_rate = miss_rate
            miss_data = self.miss.make_missdata(self.data, self.miss_rate)
            self.data_m = 1. - np.isnan(miss_data).astype(float)
            
            self.data_m_rand = binary_sampler(1-(miss_rate/10.), self.data.shape)
            self.data_m[self.data_m_rand==0.] = 0.
        self.miss_pattern = miss_pattern
        
        # sequence data
        self.ids = np.concatenate(cums)
        data_idx = np.empty((0), dtype=int)
        for i in range(1, last_cum+1):
            seq_len = (self.ids == i).sum()
            start_id = np.argmax(self.ids == i)
            # possible data number in seqeunce
            time_len = seq_len - window_size + 1
            start_ids = np.arange(start_id, start_id+time_len)
            data_idx = np.append(data_idx, start_ids)
            
        # start index set for sequence data
        self.data_idx = data_idx
        self.input_width = input_width
        self.no = len(data_idx)
        
        #print('self.no = ', self.no)
        
        self.batch_size = batch_size
        
        # random shuffling  index
        self.batch_idx = sample_batch_index(self.no, self.no)
        self.batch_id = 0
        self.shape = (batch_size,self.input_width)+self.data.shape[1:]
        #self.hint_rate = hint_rate
            
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        # Sample batch
        x = np.empty((0, self.input_width, self.data.shape[1]))
        #m = np.empty((0, self.input_width, self.data.shape[1]))
        #h = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.input_width, self.data.shape[1]))
        #print(x.shape)
        #print(self.data.shape)
        #print(self.input_width)
        #self.batch_idx = sample_batch_index(self.no, self.batch_size)
        for cnt in range(0, self.batch_size):
            i = self.batch_idx[self.batch_id]
            self.batch_id += 1
            #self.batch_id %= self.batch_size
            self.batch_id %= self.no
            if self.miss_pattern and (self.batch_id == 0):
                self.batch_idx = sample_batch_index(self.no, self.no)
                miss_data = self.miss.make_missdata(self.data, self.miss_rate)
                self.data_m = 1. - np.isnan(miss_data).astype(float)
                self.data_m_rand = binary_sampler(1-self.miss_rate/10., self.data.shape)
                self.data_m[self.data_m_rand==0.] = 0.
            idx1 = self.data_idx[i]
            idx2 = self.data_idx[i]+self.input_width
            #print(idx1, idx2)
        
            Y_mb = self.data[idx1:idx2].copy()
            X_mb = Y_mb.copy()
            M_mb = self.data_m[idx1:idx2]
            Z_mb = uniform_sampler(0, 0.01, shape=X_mb.shape)
            X_mb = M_mb*X_mb + (1-M_mb)*Z_mb
            #H_mb_temp = binary_sampler(self.hint_rate, shape=X_mb.shape)
            #H_mb = M_mb * H_mb_temp
            X_mb[M_mb == 0] = np.nan
            Y_mb[M_mb == 1] = np.nan
            x = np.append(x, [X_mb], axis=0)
            #m = np.append(m, [M_mb], axis=0)
            #h = np.append(h, [H_mb], axis=0)
            y = np.append(y, [Y_mb], axis=0)
            
        #return [x, m, h], y
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

In [224]:
class GAIN(keras.Model):
    def __init__(self, shape, alpha=100., load=False, hint_rate=0.9, gen_sigmoid=True, **kwargs):
        super(GAIN, self).__init__(**kwargs)
        self.shape = shape
        self.dim = np.prod(shape).astype(int)
        self.h_dim = self.dim
        self.gen_sigmoid = gen_sigmoid
        self.build_generator()
        self.build_discriminator()
        self.hint_rate = hint_rate
        self.alpha = alpha
        self.generator_optimizer = Adam()
        self.discriminator_optimizer = Adam()

    ## GAIN models
    def build_generator(self):
        last_activation = 'sigmoid' if self.gen_sigmoid else None
        xavier_initializer = tf.keras.initializers.GlorotNormal()

        shape = self.shape
        #x = Input(shape=(self.dim,), name='generator_input_x')
        #m = Input(shape=(self.dim,), name='generator_input_m')
        x = Input(shape=shape, name='generator_input_x')
        m = Input(shape=shape, name='generator_input_m')
        
        x_f = Flatten()(x)
        m_f = Flatten()(m)

        a = Concatenate()([x_f, m_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.dim, activation=last_activation, kernel_initializer=xavier_initializer)(a)
        G_prob = keras.layers.Reshape(shape)(a)
        self.generator = keras.models.Model([x, m], G_prob, name='generator')

    def build_discriminator(self):
        xavier_initializer = tf.keras.initializers.GlorotNormal()
        shape = self.shape

        #x = Input(shape=(self.dim,), name='discriminator_input_x')
        #h = Input(shape=(self.dim,), name='discriminator_input_h')
        x = Input(shape=shape, name='discriminator_input_x')
        h = Input(shape=shape, name='discriminator_input_h')
        
        x_f = Flatten()(x)
        h_f = Flatten()(h)

        a = Concatenate()([x_f, h_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.dim, activation='sigmoid', kernel_initializer=xavier_initializer)(a)
        D_prob = keras.layers.Reshape(shape)(a)
        self.discriminator = keras.models.Model([x, h], D_prob, name='discriminator')
        
    def call(self, inputs):
        if isinstance(inputs, tuple):
            inputs = inputs[0]
        shape = inputs.shape
        dims = np.prod(shape[1:])
        input_width = shape[1]
        #print('inputs.shape=',inputs.shape)
        x = inputs
        #x = x.reshape((n, -1))
        #print('dims=',dims)
        #x = keras.layers.Reshape((dims,))(x)
        #x = keras.layers.Reshape(tf.TensorShape((self.dim,)))(x)
        #print('x =', x)
        #print('x.shape = ', x.shape)
        #x = keras.layers.Reshape(tf.TensorShape([57]))(x)
        
        isnan = tf.math.is_nan(x)
        #m = 1.- keras.backend.cast(isnan, dtype=tf.float32)
        m = tf.where(isnan, 0., 1.)
        z = keras.backend.random_uniform(shape=tf.shape(x), minval=0.0, maxval=0.01)
        x = tf.where(isnan, z, x)
        #z = uniform_sampler(0, 0.01, shape=x.shape)
        #z = tf.keras.backend.random_uniform(shape=x.shape, minval=0.0, maxval=0.01)
        imputed_data = self.generator([x, m], training=False)
        #imputed_data = m*x + (1-m)*imputed_data
        #imputed_data = tf.where(isnan, imputed_data, np.nan)
        imputed_data = tf.where(isnan, imputed_data, x)
        #imputed_data = keras.layers.Reshape(shape[1:])(imputed_data)
        #print('imputed_data.shape = ', imputed_data.shape)
        
        return imputed_data
    
    def D_loss(M, D_prob):
        ## GAIN loss
        return -tf.reduce_mean(M * tf.keras.backend.log(D_prob + 1e-8) \
                         + (1-M) * tf.keras.backend.log(1. - D_prob + 1e-8))
    
    def G_loss(self, M, D_prob, X, G_sample):
        G_loss_temp = -tf.reduce_mean((1-M) * tf.keras.backend.log(D_prob + 1e-8))
        MSE_loss = tf.reduce_mean((M * X - M * G_sample)**2) / (tf.reduce_mean(M) + 1e-8)
        #G_loss_temp = GAIN.G_loss_bincross(M, D_prob)
        #MSE_loss = GAIN.MSE_loss(M, X, G_sample)
        G_loss = G_loss_temp + self.alpha * MSE_loss
        return G_loss
        
    def RMSE_loss(y_true, y_pred):
        isnan = tf.math.is_nan(y_true)
        M = tf.where(isnan, 1., 0.)
        return tf.sqrt(tf.reduce_sum(tf.where(isnan, 0., y_pred-y_true)**2)/tf.reduce_sum(1-M))
    
    def train_step(self, data):
        #[x, m, h], y = data
        x, y = data
        #X = keras.layers.Reshape((self.dim,), input_shape=self.shape)(x)
        #Y = keras.layers.Reshape((self.dim,), input_shape=self.shape)(y)
        #X = keras.layers.Flatten()(x)
        #Y = keras.layers.Flatten()(y)
        X = x
        Y = y
        #X = tf.reshape(x, shape=(x.shape[0], -1))
        #Y = tf.reshape(y, shape=(x.shape[0], -1))
        isnan = tf.math.is_nan(X)
        #M = 1 - keras.backend.cast(isnan, dtype=tf.float32)
        M = tf.where(isnan, 0., 1.)
        Z = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=0.01)
        #H_temp = binary_sampler(self.hint_rate, shape=X.shape)
        H_rand = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=1.)
        #H_temp = 1*keras.backend.cast((H_rand < self.hint_rate), dtype=tf.float32)
        H_temp = tf.where(H_rand < self.hint_rate, 1., 0.)
        
        H = M * H_temp + 0.5*(1-H_temp)
        #X = M * X + (1-M) * Z
        X = tf.where(isnan, Z, X)
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            G_sample = self.generator([X, M], training=True)

            # Combine with observed data
            #Hat_X = tf.where(isnan, G_sample, X)
            Hat_X = X * M + G_sample * (1-M)
            D_prob = self.discriminator([Hat_X, H], training=True)
            gen_loss = self.G_loss(M, D_prob, X, G_sample)
            disc_loss = tf.keras.backend.mean(tf.keras.losses.binary_crossentropy(M, D_prob))
            #disc_loss = GAIN.D_loss(M, D_prob)
            #disc_loss = GAIN.D_loss(M, D_prob)

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.generator_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))
        
        rmse = tf.sqrt(tf.reduce_sum(tf.where(isnan, G_sample - Y, 0.)**2)/tf.reduce_sum(1-M))
        return {
                 'gen_loss':     gen_loss,
                 'disc_loss':    disc_loss,
                 'rmse':         rmse,
               }
    
    def save(self, save_dir='save', prefix=''):
        if not os.path.exists(save_dir):
          os.makedirs(save_dir)
        disc_savefile = os.path.join(save_dir, prefix+'discriminator.h5')
        gen_savefile = os.path.join(save_dir, prefix+'generator.h5')
        self.discriminator.save_weights(disc_savefile)
        self.generator.save_weights(gen_savefile)

    def load(self, save_dir='save', prefix=''):
        disc_savefile = os.path.join(save_dir, prefix+'discriminator.h5')
        gen_savefile = os.path.join(save_dir, prefix+'generator.h5')
        try:
          self.discriminator.load_weights(disc_savefile)
          self.generator.load_weights(gen_savefile)
          print('model weights loaded')
        except:
          print('model loadinng error')

In [225]:
def make_dataset_gain(self, data):
  dg = GainDataGenerator(
      df,
      input_width = self.input_width,
      label_width = self.label_width,
      batch_size = 128,
      normalize = False,
      miss_pattern = True,
      miss_rate = 0.15,
      fill_no = 3,
  )
  self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.shape,
        dg.shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_gain

In [226]:
train_df = df_all
val_df = df_all
test_df = df_all

In [227]:
wide_window = WindowGenerator(
    input_width=24*5, label_width=24*5, shift=0,
    #label_columns=['T (degC)']
)
_ = wide_window.train
wide_window

Total window size: 120
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]
Label indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 

In [228]:
#wide_window.plot(plot_col='클로로필-a')
wide_window.plot(plot_col='클로로필-a_값')
print('make_dataset_gain: dg.no = ', wide_window.dg.no)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

make_dataset_gain: dg.no =  29385


In [229]:
val_performance = {}
performance = {}

In [230]:
gain = GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False)
gain.compile(loss=GAIN.RMSE_loss)

In [231]:
MAX_EPOCHS = 2000

def compile_and_fit(model, window, patience=10):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  #model.compile(loss=tf.losses.MeanSquaredError(),
                #optimizer=tf.optimizers.Adam(),
                #metrics=[tf.metrics.MeanAbsoluteError()])
  model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [100]:
#model.fit를 사용하지 않을 때에는 학습 데이터 로딩
#gain.load(save_dir='save')
gain.load(save_dir='save', prefix='nagdong2')

model weights loaded


In [232]:
history = compile_and_fit(gain, wide_window, patience=MAX_EPOCHS//5)


val_performance['Gain'] = gain.evaluate(wide_window.val)
performance['Gain'] = gain.evaluate(wide_window.test, verbose=0)


#early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
#                                                    patience=2,
#                                                    mode='min')
#gain.compile()

Epoch 1/2000
1/1 [==============================] - 1s 892ms/step - gen_loss: 125.3887 - disc_loss: 0.7181 - rmse: 1.1789 - val_loss: 1.0341
Epoch 2/2000
1/1 [==============================] - 0s 103ms/step - gen_loss: 102.7850 - disc_loss: 0.5209 - rmse: 1.0337 - val_loss: 0.9130
Epoch 3/2000
1/1 [==============================] - 0s 98ms/step - gen_loss: 87.8296 - disc_loss: 0.4943 - rmse: 0.9056 - val_loss: 0.8610
Epoch 4/2000
1/1 [==============================] - 0s 102ms/step - gen_loss: 83.8946 - disc_loss: 0.4010 - rmse: 0.9492 - val_loss: 0.8649
Epoch 5/2000
1/1 [==============================] - 0s 102ms/step - gen_loss: 68.9259 - disc_loss: 0.3821 - rmse: 0.8908 - val_loss: 0.7898
Epoch 6/2000
1/1 [==============================] - 0s 106ms/step - gen_loss: 64.4325 - disc_loss: 0.3905 - rmse: 0.8952 - val_loss: 0.7770
Epoch 7/2000
1/1 [==============================] - 0s 98ms/step - gen_loss: 58.7280 - disc_loss: 0.3976 - rmse: 0.8399 - val_loss: 0.7584
Epoch 8/2000
1/1 [==

1/1 [==============================] - 0s 108ms/step - gen_loss: 6.3451 - disc_loss: 0.2596 - rmse: 0.3448 - val_loss: 0.3288
Epoch 118/2000
1/1 [==============================] - 0s 117ms/step - gen_loss: 6.6791 - disc_loss: 0.2569 - rmse: 0.3589 - val_loss: 0.3149
Epoch 119/2000
1/1 [==============================] - 0s 108ms/step - gen_loss: 5.0761 - disc_loss: 0.2593 - rmse: 0.3030 - val_loss: 0.3408
Epoch 120/2000
1/1 [==============================] - 0s 99ms/step - gen_loss: 6.6183 - disc_loss: 0.2566 - rmse: 0.3767 - val_loss: 0.5514
Epoch 121/2000
1/1 [==============================] - 0s 92ms/step - gen_loss: 6.7900 - disc_loss: 0.2524 - rmse: 0.3625 - val_loss: 0.3220
Epoch 122/2000
1/1 [==============================] - 0s 107ms/step - gen_loss: 7.3515 - disc_loss: 0.2504 - rmse: 0.4011 - val_loss: 0.3283
Epoch 123/2000
1/1 [==============================] - 0s 105ms/step - gen_loss: 7.2944 - disc_loss: 0.2455 - rmse: 0.5775 - val_loss: 0.3425
Epoch 124/2000
1/1 [==========

1/1 [==============================] - 0s 105ms/step - gen_loss: 5.5474 - disc_loss: 0.2052 - rmse: 0.3683 - val_loss: 0.3430
Epoch 234/2000
1/1 [==============================] - 0s 116ms/step - gen_loss: 5.4727 - disc_loss: 0.1997 - rmse: 0.3437 - val_loss: 0.3424
Epoch 235/2000
1/1 [==============================] - 0s 101ms/step - gen_loss: 5.0924 - disc_loss: 0.2005 - rmse: 0.3009 - val_loss: 0.5592
Epoch 236/2000
1/1 [==============================] - 0s 95ms/step - gen_loss: 5.5880 - disc_loss: 0.1990 - rmse: 0.4701 - val_loss: 0.4093
Epoch 237/2000
1/1 [==============================] - 0s 102ms/step - gen_loss: 5.0517 - disc_loss: 0.2050 - rmse: 0.3043 - val_loss: 0.3322
Epoch 238/2000
1/1 [==============================] - 0s 106ms/step - gen_loss: 6.0819 - disc_loss: 0.2074 - rmse: 0.3764 - val_loss: 0.3343
Epoch 239/2000
1/1 [==============================] - 0s 111ms/step - gen_loss: 6.1862 - disc_loss: 0.1954 - rmse: 0.3682 - val_loss: 0.3344
Epoch 240/2000
1/1 [=========

1/1 [==============================] - 0s 103ms/step - gen_loss: 4.7194 - disc_loss: 0.1786 - rmse: 0.3387 - val_loss: 0.3109
Epoch 350/2000
1/1 [==============================] - 0s 101ms/step - gen_loss: 4.8448 - disc_loss: 0.1834 - rmse: 0.3049 - val_loss: 0.3196
Epoch 351/2000
1/1 [==============================] - 0s 139ms/step - gen_loss: 5.1030 - disc_loss: 0.1784 - rmse: 0.3418 - val_loss: 0.3083
Epoch 352/2000
1/1 [==============================] - 0s 102ms/step - gen_loss: 5.3690 - disc_loss: 0.1752 - rmse: 0.3449 - val_loss: 0.2994
Epoch 353/2000
1/1 [==============================] - 0s 100ms/step - gen_loss: 7.1073 - disc_loss: 0.1766 - rmse: 0.5056 - val_loss: 0.3195
Epoch 354/2000
1/1 [==============================] - 0s 96ms/step - gen_loss: 4.8891 - disc_loss: 0.1758 - rmse: 0.3239 - val_loss: 0.3433
Epoch 355/2000
1/1 [==============================] - 0s 95ms/step - gen_loss: 4.9319 - disc_loss: 0.1773 - rmse: 0.3772 - val_loss: 0.3215
Epoch 356/2000
1/1 [==========

1/1 [==============================] - 0s 108ms/step - gen_loss: 4.5144 - disc_loss: 0.1673 - rmse: 0.3544 - val_loss: 0.4637
Epoch 466/2000
1/1 [==============================] - 0s 98ms/step - gen_loss: 5.3543 - disc_loss: 0.1575 - rmse: 0.3421 - val_loss: 0.4625
Epoch 467/2000
1/1 [==============================] - 0s 109ms/step - gen_loss: 5.9988 - disc_loss: 0.1652 - rmse: 0.3549 - val_loss: 0.2949
Epoch 468/2000
1/1 [==============================] - 0s 98ms/step - gen_loss: 5.6190 - disc_loss: 0.1631 - rmse: 0.3915 - val_loss: 0.3477
Epoch 469/2000
1/1 [==============================] - 0s 105ms/step - gen_loss: 4.8952 - disc_loss: 0.1639 - rmse: 0.3311 - val_loss: 0.3213
Epoch 470/2000
1/1 [==============================] - 0s 105ms/step - gen_loss: 4.7410 - disc_loss: 0.1668 - rmse: 0.3428 - val_loss: 0.3196
Epoch 471/2000
1/1 [==============================] - 0s 93ms/step - gen_loss: 4.9592 - disc_loss: 0.1577 - rmse: 0.3557 - val_loss: 0.3296
Epoch 472/2000
1/1 [===========

...

1/1 [==============================] - 0s 109ms/step - gen_loss: 4.1927 - disc_loss: 0.1455 - rmse: 0.5555 - val_loss: 0.3745
Epoch 698/2000
1/1 [==============================] - 0s 100ms/step - gen_loss: 4.0919 - disc_loss: 0.1450 - rmse: 0.3149 - val_loss: 0.3140
Epoch 699/2000
1/1 [==============================] - 0s 100ms/step - gen_loss: 4.2988 - disc_loss: 0.1378 - rmse: 0.3351 - val_loss: 0.2991
Epoch 700/2000
1/1 [==============================] - 0s 116ms/step - gen_loss: 4.1067 - disc_loss: 0.1436 - rmse: 0.2892 - val_loss: 0.3492
Epoch 701/2000
1/1 [==============================] - 0s 113ms/step - gen_loss: 3.7659 - disc_loss: 0.1443 - rmse: 0.2946 - val_loss: 0.3548
Epoch 702/2000
1/1 [==============================] - 0s 122ms/step - gen_loss: 4.2881 - disc_loss: 0.1472 - rmse: 0.3510 - val_loss: 0.3320
Epoch 703/2000
1/1 [==============================] - 0s 112ms/step - gen_loss: 4.1430 - disc_loss: 0.1447 - rmse: 0.3317 - val_loss: 0.3276
Epoch 704/2000
1/1 [========

1/1 [==============================] - 0s 121ms/step - gen_loss: 3.7813 - disc_loss: 0.1371 - rmse: 0.2983 - val_loss: 0.3272
Epoch 814/2000
1/1 [==============================] - 0s 120ms/step - gen_loss: 3.9787 - disc_loss: 0.1392 - rmse: 0.2949 - val_loss: 0.3202
Epoch 815/2000
1/1 [==============================] - 0s 91ms/step - gen_loss: 3.8373 - disc_loss: 0.1374 - rmse: 0.3368 - val_loss: 0.2808
Epoch 816/2000
1/1 [==============================] - 0s 111ms/step - gen_loss: 4.3362 - disc_loss: 0.1417 - rmse: 0.3524 - val_loss: 0.2879
Epoch 817/2000
1/1 [==============================] - 0s 105ms/step - gen_loss: 3.5901 - disc_loss: 0.1357 - rmse: 0.3133 - val_loss: 0.2908
Epoch 818/2000
1/1 [==============================] - 0s 95ms/step - gen_loss: 3.8175 - disc_loss: 0.1382 - rmse: 0.3299 - val_loss: 0.3473
Epoch 819/2000
1/1 [==============================] - 0s 113ms/step - gen_loss: 3.9068 - disc_loss: 0.1404 - rmse: 0.3306 - val_loss: 0.3100
Epoch 820/2000
1/1 [==========

1/1 [==============================] - 0s 106ms/step - gen_loss: 4.3870 - disc_loss: 0.1281 - rmse: 0.3365 - val_loss: 0.3189
Epoch 930/2000
1/1 [==============================] - 0s 97ms/step - gen_loss: 3.7939 - disc_loss: 0.1302 - rmse: 0.3397 - val_loss: 0.3296
Epoch 931/2000
1/1 [==============================] - 0s 91ms/step - gen_loss: 3.7331 - disc_loss: 0.1342 - rmse: 0.3037 - val_loss: 0.3383
Epoch 932/2000
1/1 [==============================] - 0s 108ms/step - gen_loss: 3.4560 - disc_loss: 0.1312 - rmse: 0.2823 - val_loss: 0.3479
Epoch 933/2000
1/1 [==============================] - 0s 95ms/step - gen_loss: 4.1129 - disc_loss: 0.1286 - rmse: 0.3376 - val_loss: 0.3301
Epoch 934/2000
1/1 [==============================] - 0s 114ms/step - gen_loss: 3.6746 - disc_loss: 0.1277 - rmse: 0.3206 - val_loss: 0.3330
Epoch 935/2000
1/1 [==============================] - 0s 106ms/step - gen_loss: 3.6965 - disc_loss: 0.1329 - rmse: 0.2916 - val_loss: 0.3432
Epoch 936/2000
1/1 [===========

Epoch 1045/2000
1/1 [==============================] - 0s 93ms/step - gen_loss: 3.4201 - disc_loss: 0.1306 - rmse: 0.2971 - val_loss: 0.3319
Epoch 1046/2000
1/1 [==============================] - 0s 102ms/step - gen_loss: 4.7012 - disc_loss: 0.1243 - rmse: 0.3265 - val_loss: 0.3107
Epoch 1047/2000
1/1 [==============================] - 0s 99ms/step - gen_loss: 3.7205 - disc_loss: 0.1277 - rmse: 0.3215 - val_loss: 0.3206
Epoch 1048/2000
1/1 [==============================] - 0s 94ms/step - gen_loss: 5.2647 - disc_loss: 0.1256 - rmse: 0.3344 - val_loss: 0.3247
Epoch 1049/2000
1/1 [==============================] - 0s 104ms/step - gen_loss: 3.7156 - disc_loss: 0.1229 - rmse: 0.3103 - val_loss: 0.3410
Epoch 1050/2000
1/1 [==============================] - 0s 117ms/step - gen_loss: 3.5838 - disc_loss: 0.1266 - rmse: 0.3000 - val_loss: 0.3386
Epoch 1051/2000
1/1 [==============================] - 0s 93ms/step - gen_loss: 3.3814 - disc_loss: 0.1233 - rmse: 0.3317 - val_loss: 0.3421
Epoch 1052

Epoch 1103/2000
1/1 [==============================] - 0s 103ms/step - gen_loss: 3.8642 - disc_loss: 0.1282 - rmse: 0.3078 - val_loss: 0.3601
Epoch 1104/2000
1/1 [==============================] - 0s 92ms/step - gen_loss: 3.5620 - disc_loss: 0.1218 - rmse: 0.2911 - val_loss: 0.3027
Epoch 1105/2000
1/1 [==============================] - 0s 111ms/step - gen_loss: 3.3469 - disc_loss: 0.1227 - rmse: 0.3093 - val_loss: 0.3634
Epoch 1106/2000
1/1 [==============================] - 0s 100ms/step - gen_loss: 3.7567 - disc_loss: 0.1251 - rmse: 0.3414 - val_loss: 0.3177
Epoch 1107/2000
1/1 [==============================] - 0s 99ms/step - gen_loss: 3.5188 - disc_loss: 0.1246 - rmse: 0.3122 - val_loss: 0.3305
Epoch 1108/2000
1/1 [==============================] - 0s 131ms/step - gen_loss: 4.3669 - disc_loss: 0.1196 - rmse: 0.3138 - val_loss: 0.2986
Epoch 1109/2000
1/1 [==============================] - 0s 95ms/step - gen_loss: 3.2906 - disc_loss: 0.1242 - rmse: 0.2980 - val_loss: 0.3179
Epoch 111

In [233]:
#gain.save(save_dir='save_paldang')
#gain.save(prefix='nagdong')
#gain.save(prefix='nagdong2')
#gain.save(prefix='nagdong3')
#gain.save(prefix='geum')
#gain.save(prefix='nagdong4')
#gain.save(prefix='geum2')
#gain.save(prefix='yeong')
gain.save(prefix='han2')

In [234]:
#wide_window.plot(gain, plot_col='클로로필-a')
wide_window.plot(gain, plot_col='클로로필-a_값')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## GAIN 추론

In [235]:
total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print('unit_shape =', unit_shape)
time_seq = unit_shape[0]
print(time_seq)
n = (total_n//time_seq)*time_seq
print('n =', n)

gans = []
oris = []
for i in range(len(df)):
    x = df[i].to_numpy()
    total_n = x.shape[0]
    n = (total_n//time_seq)*time_seq
    x_block = x[0:n].copy()
    most_shape = x_block.shape
    x_block = x_block.reshape((-1,)+unit_shape)
    x_block_remain = x[-time_seq:].copy()
    remain_shape = x_block_remain.shape
    x_block_remain = x_block_remain.reshape((-1,)+unit_shape)
    y = gain.predict(x_block)
    y_remain = gain.predict(x_block_remain)
    y_gan = y.reshape(most_shape)
    y_remain_gan = y_remain.reshape(remain_shape)
    y_gan = np.append(y_gan, y_remain_gan[-(total_n-n):], axis=0)
    
    # cut off sin, cos data
    if (i > 0):
        x = x[:, :-4]
        y_gan = y_gan[:, :-4]
    gans.append(y_gan)
    oris.append(x)
    print(y_gan.shape)

199368
unit_shape = (120, 12)
120
n = 199320
(66456, 12)
(66456, 8)
(66456, 8)


In [236]:
gan_dfs = []
ori_dfs = []
for i in range(len(gans)):
    gan_dfs.append(pd.DataFrame(gans[i], columns=df[i].columns[:(gans[i].shape[1])]))
    ori_dfs.append(pd.DataFrame(oris[i], columns=df[i].columns[:(gans[i].shape[1])]))

In [237]:
gan_dfs[2]

,tmpr_value,ph_value,do_value,ec_value,toc_value,총질소_값,총인_값,클로로필-a_값
0,-1.525069,0.457254,0.916653,-0.487030,-1.351119,1.651509,-0.050892,-0.457194
1,-1.614207,0.461133,1.035451,-0.498397,-1.451491,1.752216,0.001234,-0.489992
2,-1.683899,0.458390,1.144923,-0.509874,-1.460901,1.680406,0.040724,-0.551025
3,-1.739459,0.447485,1.238064,-0.522330,-1.450071,1.656453,-0.354171,-0.444029
4,-1.741106,0.453409,1.256035,-0.500192,-1.429913,1.720460,-0.433150,-0.472330
...,...,...,...,...,...,...,...,...
66451,0.958259,1.210815,-0.851978,0.414090,0.735033,0.195567,-0.136578,0.138059
66452,0.974798,0.835233,-0.952717,0.113862,0.917022,0.219984,-0.204194,-0.239982
66453,0.996920,0.947258,-0.980356,0.211673,0.594085,0.273183,-0.269861,-0.175719
66454,1.191137,0.765433,-0.974723,0.275374,-0.062771,0.206697,-0.039048,-0.420308


**Save data**

In [241]:
folder = 'data'
#prefix = 'nagdong2'
#prefix = 'han'
#prefix = 'nagdong3'
#prefix = 'geum'
#prefix = 'nagdong4'
#prefix = 'geum2'
#prefix = 'yeong'
prefix = 'han2'
def save_gan_data(gan_dfs, ori_dfs, gan_files, ori_files, train_mean, train_std, prefix):
    for i in range(len(gan_dfs)):
        path = os.path.join(folder, gan_files[i])
        gan_dfs[i].to_excel(path, index=False)
        print(path)
        path = os.path.join(folder, ori_files[i])
        ori_dfs[i].to_excel(path, index=False)
    path = os.path.join(folder, prefix + '_std.xlsx')
    train_std.to_frame(name='std').to_excel(path)
    path = os.path.join(folder, prefix + '_mean.xlsx')
    train_mean.to_frame(name='mean').to_excel(path)

def load_gan_data(gan_files, ori_files, prefix):
    gan_dfs = []
    ori_dfs = []
    for i in range(len(gan_files)):
        path = os.path.join(folder, gan_files[i])
        gan_dfs.append(pd.read_excel(path))
        print(path)
        path = os.path.join(folder, ori_files[i])
        ori_dfs.append(pd.read_excel(path))
    path = os.path.join(folder, prefix + '_std.xlsx')
    train_std = pd.read_excel(path, index_col=0).loc[:, 'std']
    path = os.path.join(folder, prefix + '_mean.xlsx')
    train_mean = pd.read_excel(path, index_col=0).loc[:, 'mean']
    return gan_dfs, ori_dfs, train_mean, train_std

In [239]:
gan_dfs[2]

,tmpr_value,ph_value,do_value,ec_value,toc_value,총질소_값,총인_값,클로로필-a_값
0,-1.525069,0.457254,0.916653,-0.487030,-1.351119,1.651509,-0.050892,-0.457194
1,-1.614207,0.461133,1.035451,-0.498397,-1.451491,1.752216,0.001234,-0.489992
2,-1.683899,0.458390,1.144923,-0.509874,-1.460901,1.680406,0.040724,-0.551025
3,-1.739459,0.447485,1.238064,-0.522330,-1.450071,1.656453,-0.354171,-0.444029
4,-1.741106,0.453409,1.256035,-0.500192,-1.429913,1.720460,-0.433150,-0.472330
...,...,...,...,...,...,...,...,...
66451,0.958259,1.210815,-0.851978,0.414090,0.735033,0.195567,-0.136578,0.138059
66452,0.974798,0.835233,-0.952717,0.113862,0.917022,0.219984,-0.204194,-0.239982
66453,0.996920,0.947258,-0.980356,0.211673,0.594085,0.273183,-0.269861,-0.175719
66454,1.191137,0.765433,-0.974723,0.275374,-0.062771,0.206697,-0.039048,-0.420308


In [242]:
save_gan_data(gan_dfs, ori_dfs, gan_files, ori_files, train_mean, train_std, prefix=prefix)

data/가평2_gan.xlsx
data/의암호2_gan.xlsx
data/서상2_gan.xlsx


**Load GAN data**

In [507]:
gan_dfs2, ori_dfs2, train_mean2, train_std2 = load_gan_data(gan_files, ori_files, prefix=prefix)

data/도개_gan.xlsx
data/신암_gan.xlsx
data/회상_gan.xlsx


In [23]:
prefix = 'nagdong3'
gan_files = [ '도개3_gan.xlsx', '신암3_gan.xlsx']
ori_files = [ '도개3_org.xlsx', '신암3_org.xlsx']
auto_cols = ['tmpr_value', 'ph_value', 'do_value', 'ec_value', 'toc_value', '총질소_값', '총인_값', '클로로필-a_값']
gan_dfs, ori_dfs, train_mean, train_std = load_gan_data(gan_files, ori_files, prefix=prefix)

data/도개3_gan.xlsx
data/신암3_gan.xlsx


In [137]:
gan_nan_dfs = []
for i in range(len(gan_dfs)):
    gan_nan_dfs.append(gan_dfs[i].copy())
    isnan = ori_dfs[i].isna()
    gan_nan_dfs[i][~isnan] = np.nan

In [138]:
n = len(auto_cols)
for j in range(len(gan_dfs)):
    plt.figure(figsize=(9,20))
    for i in range(n):
        #plt.subplot('%d1%d'%(n,i))
        plt.subplot((n*100)+11+i)
        plt.plot(ori_dfs[j].iloc[:, i])
        plt.plot(gan_nan_dfs[j].iloc[:, i])
    plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
file_names = []
file_names = ['보_구미보_2016.xlsx', '보_구미보_2017.xlsx', '보_구미보_2018.xlsx', '보_구미보_2019.xlsx']
folder = 'data'
bo_dfs = []
for file_name in file_names:
    path = os.path.join(folder, file_name)
    print(path)
    bo_dfs.append(pd.read_excel(path).iloc[:, 2:4])
bo_df = pd.concat(bo_dfs, axis=0)

bo_mean = bo_df.mean()
bo_std = bo_df.std()
bo_df = (bo_df - bo_mean)/bo_std

bo_df.reset_index(drop=True, inplace=True)

data/보_구미보_2016.xlsx
data/보_구미보_2017.xlsx
data/보_구미보_2018.xlsx
data/보_구미보_2019.xlsx


In [44]:
file_names = []
file_names = ['AWS_옥산_2016.xlsx', 'AWS_옥산_2017.xlsx', 'AWS_옥산_2018.xlsx', 'AWS_옥산_2019.xlsx']
folder = 'data'
aws_dfs = []
for file_name in file_names:
    path = os.path.join(folder, file_name)
    print(path)
    aws_dfs.append(pd.read_excel(path).iloc[:, -4:])
aws_df = pd.concat(aws_dfs, axis=0)

aws_mean = aws_df.mean()
aws_std = aws_df.std()
aws_df = (aws_df - aws_mean)/aws_std
aws_df.reset_index(drop=True, inplace=True)

data/AWS_옥산_2016.xlsx
data/AWS_옥산_2017.xlsx
data/AWS_옥산_2018.xlsx
data/AWS_옥산_2019.xlsx


In [95]:
mean = pd.concat([train_mean, bo_mean, aws_mean]+qm_mean)
std = pd.concat([train_std, bo_std, aws_std]+qm_std)
mean

tmpr_value      1.619773e+01
ph_value        7.957707e+00
do_value        1.028830e+01
ec_value        2.361925e+02
toc_value       2.949259e+00
총질소_값           2.511900e+00
총인_값            1.923296e-02
클로로필-a_값        1.821510e+01
Day sin         6.037103e-15
Day cos         2.747682e-15
Year sin       -5.681266e-08
Year cos        2.053380e-05
upstrim_wlv     3.246092e+03
dwstrm_wlv      2.571552e+03
rn60m_value     7.638889e-02
ta_value        8.758325e+00
hm_value        2.880299e+01
ps_value        7.607794e+02
nh3n            7.441017e-02
no3n            1.995208e+00
ph              7.986643e+00
cod             5.426084e+00
ec              2.315260e+02
bod             1.595262e+00
ss              6.677707e+00
do              1.144842e+01
tn              2.423511e+00
tp              2.772457e-02
chlorophylla    1.183009e+01
toc             2.633367e+00
nh3n            2.100124e-01
no3n            1.963437e+00
ph              7.703737e+00
cod             3.973896e+00
ec            

# 상관관계 분석

In [96]:
from tqdm import tqdm

**Feature combine**

In [78]:
#ori_df = pd.concat(ori_dfs, axis=1)
#gan_df = pd.concat(gan_dfs, axis=1)

#ori_df = pd.concat([ori_df, bo_df, aws_df]+qm_dfs, axis=1)
#gan_df = pd.concat([gan_df, bo_df, aws_df]+qm_dfs, axis=1)
#ori_df = pd.concat(ori_dfs+qm_dfs, axis=1)
#gan_df = pd.concat(gan_dfs+qm_dfs, axis=1)
ori_df = pd.concat(ori_dfs+qm_dfs+tm_dfs, axis=1)
gan_df = pd.concat(gan_dfs+qm_dfs+tm_dfs, axis=1)

total_no = ori_df.shape[0]
train_no = int(total_no*0.7)

train_slice = slice(0, train_no)
val_slice = slice(train_no, None)
test_slice = slice(0, None)

train_df = pd.DataFrame(gan_df[train_slice])
val_df = pd.DataFrame(gan_df[val_slice])
test_df = pd.DataFrame(gan_df[test_slice])

train_ori_df = pd.DataFrame(ori_df[train_slice])
val_ori_df = pd.DataFrame(ori_df[val_slice])
test_ori_df = pd.DataFrame(ori_df[test_slice])

num_features = train_df.shape[1]
#out_num_features = oris[0].shape[1]-4
out_features = [6]
out_num_features = len(out_features)
out_num_features

1

**지점간 동일 측정항목 비교**

In [24]:
total_lag = 24*5
#total_lag = 5
#index1 = 3 # 가평 chl-a
#label1 = '가평 chl-a'
#index2 = 3+11 # 의암호 chl-a
#label2 = '의암호 chl-a'
n_col = len(auto_cols)

data = gan_df

# source 0,1,2
source = 1

n = data.shape[0]

corr = np.empty((n_col, total_lag,))
#for col in range(1):
for col in range(n_col):
#for col in range(2):
    for lag in tqdm(range(total_lag)):
        index1 = col
        index2 = col+4+(source+1)*n_col
        x1 = data.iloc[:n-lag, index1].to_numpy()
        x2 = data.iloc[lag:, index2].to_numpy()
        #df = pd.DataFrame([x1,x2], index=[index1, index2])
        #df = df.transpose()
        #df_corr = df.corr(method='pearson')
        #corr[col,lag] = df_corr.iloc[0,1]
        correl = np.corrcoef(x1, x2)
        corr[col,lag] = correl[0,1]
        

100%|██████████| 120/120 [00:00<00:00, 1369.67it/s]


In [25]:
cols = ori_df.columns
plt.figure(figsize=(9, 12))
for i in range(n_col):
    plt.subplot(n_col, 1, i+1)
    plt.plot(corr[i], label=cols[i])
    plt.legend(prop=fprop)
plt.show()
#plt.plot(corr[0], label=cols[0])
#plt.plot(corr[1], label=cols[1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
cols = ori_df.columns
plt.figure(figsize=(9, 12))
for i in range(n_col):
    plt.subplot(n_col, 1, i+1)
    plt.plot(corr[i], label=cols[i])
    plt.legend(prop=fprop)
plt.show()
#plt.plot(corr[0], label=cols[0])
#plt.plot(corr[1], label=cols[1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Source 측정요소와 Target 측정요소 간의 correlation 분석**

In [79]:
total_lag = 24*20
#total_lag = 5
#index1 = 3 # 가평 chl-a
#label1 = '가평 chl-a'
#index2 = 3+11 # 의암호 chl-a
#label2 = '의암호 chl-a'

data = gan_df

# target index
target_index = 7
# source index
n_col = data.shape[1]

n = data.shape[0]

corr = np.empty((n_col, total_lag,))
#for col in range(1):
for col in range(n_col):
#for col in range(1):
    for lag in tqdm(range(total_lag)):
        index1 = target_index
        index2 = col
        x1 = data.iloc[:n-lag, index1].to_numpy()
        x2 = data.iloc[lag:, index2].to_numpy()
        #df = pd.DataFrame([x1,x2], index=[index1, index2])
        #df = df.transpose()
        #df_corr = df.corr(method='pearson')
        #corr[col,lag] = df_corr.iloc[0,1]
        correl = np.corrcoef(x1, x2)
        corr[col,lag] = correl[0, 1]

100%|██████████| 480/480 [00:00<00:00, 1657.92it/s]


In [80]:
cols = ori_df.columns
plt.figure(figsize=(9, 50))
for i in range(n_col):
    plt.subplot(n_col, 1, i+1)
    plt.plot(corr[i], label=cols[i])
    plt.legend(prop=fprop)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [81]:
plt.figure()
plt.plot(ori_df.iloc[:, 6])
plt.plot(ori_df.iloc[:, 17])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Random forest와 Boruta를 이용한 feature 분석

**source-target data creation (target-가평, source-의암호) or (해평, 도개)**

In [82]:
hours = 24
test_avg = test_df.groupby(test_df.index //24).mean()
total_no= test_avg.shape[0]
train_no = int(total_no*0.7)

train_slice = slice(0, train_no)
val_slice = slice(train_no, None)
test_slice = slice(0, None)
train_avg = test_avg[train_slice]
val_avg = test_avg[val_slice]
#val_avg_df = pd.DataFrame(gan[val_slice], columns=columns)
#test_avg_df = pd.DataFrame(gan[test_slice], columns=columns)


In [83]:
val_avg

,tmpr_value,ph_value,do_value,ec_value,toc_value,총질소_값,총인_값,클로로필-a_값,Day sin,Day cos,...,iem_ph4,iem_doc4,iem_bod4,iem_cod4,iem_ss4,iem_ec4,iem_tn4,iem_tp4,iem_chla4,iem_toc4
1502,-1.375016,-0.131265,1.234207,0.696571,-1.177807,1.439949,-0.748309,1.424968,3.025011e-13,-3.017956e-13,...,-1.073376,1.517370,0.026563,-1.084242,-0.425245,0.344176,0.975465,-0.359533,-0.247974,-0.564965
1503,-1.399245,-0.234437,1.336220,0.637483,-1.064931,1.362769,-0.702867,1.302678,3.025011e-13,-3.017586e-13,...,-1.030719,1.517370,0.122660,-1.039073,-0.362371,0.247477,0.935564,-0.266668,-0.153092,-0.536423
1504,-1.369312,-0.559080,1.417223,0.612401,-1.060158,1.361606,-0.645016,1.244087,-5.549612e-13,5.557036e-13,...,-0.987947,1.517370,0.212656,-0.992813,-0.299329,0.150519,0.895555,-0.175090,-0.057955,-0.510570
1505,-1.328711,-0.847352,1.244137,0.663418,-1.068044,1.163487,-0.661874,1.053149,3.025011e-13,-3.017956e-13,...,-0.950544,1.517370,0.287850,-0.949559,-0.244199,0.065730,0.860568,-0.095852,0.025241,-0.489486
1506,-1.293919,-1.002571,1.139609,0.695144,-0.950244,1.061592,-0.634158,0.958409,3.025011e-13,-3.017586e-13,...,-0.923991,1.517370,0.339543,-0.913410,-0.205061,0.005537,0.835730,-0.040007,0.084304,-0.475253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2141,-0.425117,-0.090468,-0.232479,0.023836,0.584509,0.009591,0.001753,0.009868,3.025011e-13,5.557036e-13,...,-0.115491,1.758127,-0.859245,-0.781950,-0.437952,-1.264009,0.504138,-0.461547,0.056884,-0.933179
2142,-0.425152,0.033547,-0.208569,0.072452,0.607814,0.140800,0.020031,0.146335,3.025011e-13,-3.017586e-13,...,-0.184041,1.766245,-1.010180,-0.863238,-0.469760,-1.309591,0.552205,-0.461955,-0.118040,-1.007771
2143,-0.444751,0.121109,-0.145324,0.009237,0.703754,0.129770,-0.091475,0.431187,-5.549612e-13,-3.017956e-13,...,-0.255449,1.772357,-1.167406,-0.947913,-0.502894,-1.357074,0.602276,-0.462253,-0.300255,-1.085471
2144,-0.447053,0.332158,-0.018808,0.001368,0.702019,0.058706,-0.451500,1.055650,-5.549612e-13,5.557036e-13,...,-0.329771,1.776440,-1.331051,-1.036045,-0.537381,-1.406495,0.654391,-0.462445,-0.489909,-1.166343


## Data Generator

In [32]:
from tqdm import tqdm

In [84]:
class RFDataGenerator(keras.utils.Sequence):
    'Generates data for water'
    def __init__(self,
                 imputed_data,
                 ori_data = None,
                 batch_size = None,
                 input_width = 24*3,
                 label_width = 1,
                 shift = 24*1,
                 skip_time = None,
                 shuffle = False,
                 out_features = None,
                 out_num_features = 1,
                 selected_features = None,
                ):
        'Initialization'
        self.window_size = input_width+shift
        self.total_no = imputed_data.shape[0]
        self.data = imputed_data
        self.input_width = input_width
        self.label_width = label_width
        self.input_shape = (batch_size, input_width, self.data.shape[1])
        self.out_num_features = out_num_features
        if out_features:
            self.out_features = out_features
        else:
            self.out_features = [i for i in range(out_num_features)]
        self.label_shape = (batch_size, label_width, self.out_num_features)
        if (skip_time):
            # TO-DO
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
        else:
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
            
        if batch_size:
            self.batch_size = batch_size
        else:
            self.batch_size = self.no
            
        if shuffle:
            self.batch_idx = np.random.permutation(self.no)
        else:
            self.batch_idx = np.arange(0, self.no)
        self.batch_id = 0
        self.selected_features = selected_features
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        #print('self.no =', self.no)
        #print('self.total_no =', self.total_no)
        #print('self.batch_id =', self.batch_id)
        # Sample batch
        label_width = self.label_width
        batch_idx = self.batch_idx
        
        #x = np.empty((0, self.input_width, self.data.shape[1]))
        self.derived_column_features = 6
        if self.selected_features is None: # all features
            x = np.empty((self.batch_size, (self.input_width+self.derived_column_features)*self.data.shape[1]))
        else:
            x = np.empty((self.batch_size, np.sum(boruta_features.astype(int))))
        y = np.empty((self.batch_size,))
        for cnt in tqdm(range(0, self.batch_size)):
            i = self.batch_id
            self.batch_id += 1
            idx1 = self.data_idx[batch_idx[i]]
            idx2 = idx1 + self.input_width
            
            X = self.data[idx1:idx2]
            
            idx1 = self.data_idx[batch_idx[i]] + self.window_size - label_width
            idx2 = idx1 + label_width
            
            #Y = self.data[idx1:idx2,:,:out_num_features]
            Y = self.data.iloc[idx1:idx2, self.out_features]
            #print('Y.shape = ', Y.shape)
            #Y = Y.iloc[:,:out_num_features]
            avg = X.mean(axis=0)
            avg.name = 'avg'
            val_min = X.min(axis=0)
            val_min.name = 'min'
            val_max = X.max(axis=0)
            val_max.name = 'max'
            std = X.std(axis=0)
            std.name = 'std'
            last_low = X[-1:] - std
            last_low.name = 'low'
            last_high = X[-1:] + std
            last_high.name = 'high'
            
            #X = pd.concat([X, avg])
            X = X.append(avg)
            X = X.append(val_min)
            X = X.append(val_max)
            X = X.append(std)
            X = X.append(last_low)
            X = X.append(last_high)
            #print('X.shape = ', X.shape)
            #print('x.shape = ', x.shape)
            #X = X.to_numpy().reshape(self.input_width*self.data.shape[1])
            X = X.to_numpy().reshape((-1,))
            if self.selected_features is not None:
                X = X[self.selected_features]
            #Y = Y.to_numpy().reshape(self.label_width*self.out_num_features)
            
            self.batch_id %= self.no
            
            #x = np.append(x, [X], axis = 0)
            #y = np.append(y, [Y], axis = 0)
                
            x[cnt] = X
            y[cnt] = Y.iloc[0]
            
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

**예측 요소 선정**

In [85]:
# 7: chl-a
# 4: TOC
out_features = [7]
# days to use
input_days = 14

In [86]:
# daily averaged data set
rf_train = RFDataGenerator(train_avg, input_width=input_days, shift=5, out_features=out_features)
rf_test = RFDataGenerator(val_avg, input_width=input_days, shift=5, out_features=out_features)

In [87]:
#rf_train = RFDataGenerator(train_df, shift=24*5, out_features=out_features)
#rf_test = RFDataGenerator(val_df, shift=24*5, out_features=out_features)

In [88]:
it = iter(rf_train)
x_train, y_train = next(it)
it = iter(rf_test)
x_test, y_test = next(it)

100%|██████████| 625/625 [00:04<00:00, 146.59it/s]


## Boruta

In [38]:
from sklearn.ensemble import RandomForestRegressor

In [89]:
# n_jobs = CPU core number
forest = RandomForestRegressor(n_estimators=100, n_jobs=16, verbose=1)

In [90]:
forest.fit(x_train, y_train)

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    3.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:   11.3s finished


RandomForestRegressor(n_jobs=16, verbose=1)

In [91]:
#print("Score without interactions: {:.3f}".format(
#    rf.score(x_test, y_test)))
print("Accuracy on training set: {:.3f}".format(forest.score(x_train, y_train)))
print("Accuracy on test set: {:.3f}".format(forest.score(x_test, y_test)))

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.1s finished


Accuracy on training set: 0.980
Accuracy on test set: 0.158


In [92]:
importances = forest.feature_importances_
feature2d = importances.reshape((rf_train.input_width+rf_train.derived_column_features,-1))
feature2d_nan = feature2d.copy()
feature2d_nan[feature2d == 0.] = np.nan
feature2d_log = np.log(feature2d_nan)

In [93]:
# create labels
time_labels = []

for i in range(rf_train.input_width):
    time_labels.append('t-%d'%(-i+rf_train.input_width-1) )
time_labels.append('avg')
time_labels.append('min')
time_labels.append('max')
time_labels.append('std')
time_labels.append('low')
time_labels.append('high')
column_labels = test_avg.columns.to_list()
for i in range(7):
    column_labels[i] = 'T'+column_labels[i]
    column_labels[i+11] = 'S'+column_labels[i+11]

feature_labels = []
for j in range(len(time_labels)):
    for i in range(test_avg.shape[1]):
        feature_labels.append(time_labels[j]+' '+column_labels[i])
feature_labels_np = np.array(feature_labels)

In [94]:
fig, ax = plt.subplots(figsize=(9,7))
ax.imshow(feature2d_log, cmap='rainbow')
# We want to show all ticks...
ax.set_xticks(np.arange(len(column_labels)))
ax.set_yticks(np.arange(len(time_labels)))
# ... and label them with the respective list entries
ax.set_xticklabels(column_labels, fontproperties=fprop)
ax.set_yticklabels(time_labels, fontproperties=fprop)
# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
# Loop over data dimensions and create text annotations.
'''
for i in range(len(time_labels)):
    for j in range(len(column_labels)):
        text = ax.text(j, i, '%.3f'%feature2d[i, j],
                       ha="center", va="center", color="w")
'''
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Tree’s Feature Importance from Mean Decrease in Impurity (MDI)**

In [60]:
sorted_idx = importances.argsort()
fig, ax = plt.subplots(figsize=(8,8))
n = 30
#y_ticks = np.arange(0, len(feature_names))
y_ticks = np.arange(0, n)
ax.barh(y_ticks, importances[sorted_idx[-n:]])
ax.set_yticks(y_ticks)
ax.set_yticklabels(feature_labels_np[sorted_idx[-n:]], font_properties=fprop)
fig.tight_layout()
plt.suptitle("Random Forest Feature Importances (MDI)")
plt.show()

#feature_labels_np[sorted_idx[-n:]]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [143]:
from boruta import BorutaPy

In [144]:
boruta = BorutaPy(
   estimator = forest, 
   n_estimators = 'auto',
   max_iter = 100 # number of trials to perform
)

**Boruta 학습**

In [146]:
boruta.fit(x_train, y_train)

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    1.8s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:   12.7s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:   30.8s
[Parallel(n_jobs=16)]: Done 481 out of 481 | elapsed:   35.1s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    1.7s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:   12.6s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:   30.8s
[Parallel(n_jobs=16)]: Done 481 out of 481 | elapsed:   35.1s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    1.7s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:   12.6s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:   30.5s
[Parallel(n_jobs=16)]: Done 481 out of 481 | elapsed: 

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.4s
[Parallel(n_jobs=16)]: Done 196 out of 196 | elapsed:    2.6s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.3s
[Parallel(n_jobs=16)]: Done 196 out of 196 | elapsed:    2.5s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.3s
[Parallel(n_jobs=16)]: Done 196 out of 196 | elapsed:    2.6s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.3s
[Parallel(n_jobs=16)]: Done 196 out of 196 | elapsed:    2.6s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0

[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.4s
[Parallel(n_jobs=16)]: Done 194 out of 194 | elapsed:    2.5s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.3s
[Parallel(n_jobs=16)]: Done 194 out of 194 | elapsed:    2.6s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.3s
[Parallel(n_jobs=16)]: Done 194 out of 194 | elapsed:    2.5s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.3s
[Parallel(n_jobs=16)]: Done 194 out of 194 | elapsed:    2.5s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.4s
[Parallel(n_jobs=16)]: Done 194 out of 194 | elapsed:    2.5s finished
[Parall

BorutaPy(estimator=RandomForestRegressor(n_estimators=194, n_jobs=16,
                                         random_state=RandomState(MT19937) at 0x7FABCB4F7160,
                                         verbose=1),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x7FABCB4F7160)

**Boruta feature map**

In [147]:
green_img = boruta.support_.astype(int).reshape((-1, train_avg.shape[1]))
blue_img = boruta.support_weak_.astype(int).reshape((-1, train_avg.shape[1]))

In [148]:
### print results
#green_area = test_avg.columns[boruta.support_].to_list()
#blue_area = test_avg.columns[boruta.support_weak_].to_list()
#print('features in the green area:', green_area)
#print('features in the blue area:', blue_area)

In [149]:
fig, ax = plt.subplots(figsize=(9,5))
plt.title('Green Features')
plt.imshow(green_img, cmap='Greens')
# We want to show all ticks...
ax.set_xticks(np.arange(len(column_labels)))
ax.set_yticks(np.arange(len(time_labels)))
# ... and label them with the respective list entries
ax.set_xticklabels(column_labels, fontproperties=fprop)
ax.set_yticklabels(time_labels, fontproperties=fprop)
# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
plt.show()
fig, ax = plt.subplots(figsize=(9,5))
plt.title('Blue Features')
plt.imshow(blue_img, cmap='Blues')
# We want to show all ticks...
ax.set_xticks(np.arange(len(column_labels)))
ax.set_yticks(np.arange(len(time_labels)))
# ... and label them with the respective list entries
ax.set_xticklabels(column_labels, fontproperties=fprop)
ax.set_yticklabels(time_labels, fontproperties=fprop)
# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/kotech/workspace/venv-tensor2n-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Random forest

In [150]:
#boruta_features = boruta.support_ | boruta.support_weak_
boruta_features = boruta.support_

In [151]:
np.sum(boruta_features.astype(int))

163

In [152]:
# daily averaged data set
rf_train = RFDataGenerator(train_avg, input_width=input_days, shift=5, out_features=out_features,
                           selected_features=boruta_features)
rf_test = RFDataGenerator(val_avg, input_width=input_days, shift=5, out_features=out_features,
                          selected_features=boruta_features)

In [153]:
it = iter(rf_train)
x_train, y_train = next(it)
it = iter(rf_test)
x_test, y_test = next(it)

100%|██████████| 420/420 [00:11<00:00, 35.48it/s]


In [154]:
forest.fit(x_train, y_train)

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.2s
[Parallel(n_jobs=16)]: Done 194 out of 194 | elapsed:    1.1s finished


RandomForestRegressor(n_estimators=194, n_jobs=16,
                      random_state=RandomState(MT19937) at 0x7FABCB4F7160,
                      verbose=1)

In [155]:
#print("Score without interactions: {:.3f}".format(
#    rf.score(x_test, y_test)))
print("Accuracy on training set: {:.3f}".format(forest.score(x_train, y_train)))
print("Accuracy on test set: {:.3f}".format(forest.score(x_test, y_test)))

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 194 out of 194 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 194 out of 194 | elapsed:    0.1s finished


Accuracy on training set: 0.984
Accuracy on test set: 0.255


In [156]:
y_pred = forest.predict(x_test)

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 194 out of 194 | elapsed:    0.1s finished


In [978]:
std = train_std[out_features[0]]
mean = train_mean[out_features[0]]
y_test_denorm = y_test*std + mean
y_pred_denorm = y_pred*std + mean

In [979]:
norm_mae = np.mean(np.abs(y_pred - y_test))
print('norm_mae =', norm_mae)
mae = np.mean(np.abs(y_pred_denorm - y_test_denorm))
print('mae =', mae)

norm_mae = 0.47531242982703936
mae = 6.905594590176136


In [980]:
plt.figure()
plt.plot(y_test_denorm)
plt.plot(y_pred_denorm)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [981]:
y_pred_train = forest.predict(x_train)

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 182 out of 182 | elapsed:    0.1s finished


In [982]:
y_train_denorm = y_train*std + mean
y_pred_train_denorm = y_pred_train*std + mean
plt.figure()
plt.plot(y_train_denorm)
plt.plot(y_pred_train_denorm)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [983]:
plt.figure()
plt.plot(y_train)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Original hour data plot**

In [728]:
chl_a = ori_df.iloc[:, 6]
plt.figure()
plt.plot(chl_a)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# 연습섹션

In [635]:
from sklearn.metrics import r2_score
#from scipy import stats
from sklearn.linear_model import LinearRegression

#creating data
x = np.array([0,1,2,3,4,5,6,7,8,9])
y = np.array([0,2,3,5,8,13,21,34,55,89])

#creating OLS regression
slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)

X = x.reshape((-1, 1))

lr = LinearRegression()
lr.fit(X,y)
y_pred = lr.predict(X)
#def linefitline(b):
#    return intercept + slope * b
#line1 = linefitline(x)

#plot line
plt.figure()
plt.scatter(x,y)
plt.scatter(x,y_pred)
#plt.plot(x,line1, c = 'g')
plt.show()

accuracy = lr.score(X, y)
print('accuracy =', accuracy)
correl = np.corrcoef(y, y_pred)
print('correl =')
print(correl)
print('r2 =', correl[0,1]*correl[0,1])

from sklearn.metrics import r2_score 
R_square = r2_score(y, y_pred) 
print('Coefficient of Determination', R_square) 


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

accuracy = 0.7670001449905756
correl =
[[1.         0.87578544]
 [0.87578544 1.        ]]
r2 = 0.7670001449905752
Coefficient of Determination 0.7670001449905756


least square method에서는 correl^2과 R^2가 동일하다.  
하지만 다른 경우라면...

In [637]:
y = np.array([0,2,3,5,8,13,21,34,55,89])
y_pred = np.array([1,2,7,5,5,12,21,34,55,30])
correl = np.corrcoef(y, y_pred)
print('correl =')
print(correl)
print('correl**2 =', correl[0,1]*correl[0,1])

from sklearn.metrics import r2_score 
R_square = r2_score(y, y_pred) 
print('Coefficient of Determination', R_square) 

correl =
[[1.         0.77879727]
 [0.77879727 1.        ]]
correl**2 = 0.6065251885388013
Coefficient of Determination 0.533758639021797
